<a href="https://colab.research.google.com/github/bhandrigan/ag_trends/blob/main/ag_trends_us_states.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytrends google-ads scrapingbee  shapely>=2.0.0 -q

In [ ]:
# @title Load libraries and functions
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import requests
import gzip
from io import BytesIO
from pytrends.request import TrendReq
from google.ads.googleads.client import GoogleAdsClient
from google.api_core.exceptions import ResourceExhausted
from requests.exceptions import RequestException
from requests.exceptions import ReadTimeout
import time
import os
import zipfile
from urllib.parse import urlparse
import tempfile
import json
from google.colab import userdata
import zipfile
import re
import random
from scrapingbee import ScrapingBeeClient
import shapely
from shapely.geometry import shape
import multiprocessing as mp
import unicodedata
import matplotlib.ticker as ticker
from stats_can import StatsCan
from census import Census
import us
import gc

# Initialize the Census API
CENSUS_API_KEY = userdata.get('CENSUS_API')
c = Census(CENSUS_API_KEY, year=2022)  # Update year to latest ACS available if needed
sc = StatsCan()




scrapingbee_api_key = userdata.get('SCRAPING_BEE')
gads_sa = json.loads(userdata.get('N90_GADS_SA_ACCOUNT_JSON'))
gads_api_key = userdata.get('N90_GADS_API_KEY')
sa_account = tempfile.NamedTemporaryFile(delete=False)
sa_account.write(json.dumps(gads_sa).encode())
sa_account.close()
sa_account_path = sa_account.name
search_api_io_key = userdata.get('SEARCH_API_IO_KEY')

gads_account = "8417741864"
use_proto_plus = True
impersonated_email = "api@n90.co"


# create google-ads.yaml file text
yaml_content = f"""
developer_token: {gads_api_key}
use_proto_plus: {use_proto_plus}
json_key_file_path: {sa_account_path}
impersonated_email: {impersonated_email}
login_customer_id: {gads_account}
"""

# Initialize the client with the dictionary configuration (hypothetical)
client = GoogleAdsClient.load_from_string(yaml_content)


def normalize_text(text):
    text = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in text if not unicodedata.combining(c)).upper()

def fetch_unzip_load_shapefile_flexible(url, output_dir):
    """
    Fetches a zip file containing shapefiles from a URL,
    unzips it if needed, and loads the first .shp file found
    in the output directory into a GeoDataFrame.

    Removes query string variables from the local filename.

    Args:
        url (str): The URL of the zip file containing the shapefiles.
        output_dir (str): The directory where the zip file will be downloaded
                          and unzipped.

    Returns:
        gpd.GeoDataFrame: The loaded shapefile as a GeoDataFrame, or None if
                          download, unzipping, or loading fails, or if no
                          .shp files are found.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Parse the URL to get the path component
    parsed_url = urlparse(url)
    # Get the base filename from the URL path and remove query string
    zip_filename = os.path.basename(parsed_url.path)
    zip_file_path = os.path.join(output_dir, zip_filename)

    # Check if the zip file already exists (indicates previous download/unzip)
    if os.path.exists(zip_file_path):
        print(f"Zip file '{zip_filename}' already exists in '{output_dir}'. Skipping download.")
    else:
        # If the zip file doesn't exist, attempt to download
        print(f"Zip file '{zip_filename}' not found. Attempting to download from {url}...")
        try:
            response = requests.get(url, stream=True, verify=False)
            response.raise_for_status()  # Raise an HTTPError for bad responses

            # Download the zip file
            with open(zip_file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Zip file downloaded to '{zip_file_path}'.")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading file: {e}")
            return None

    # Check if .shp files already exist in the output directory
    shp_files = [f for f in os.listdir(output_dir) if f.endswith('.shp')]
    if shp_files:
        print(f"Shapefile(s) found in '{output_dir}'. Loading the first one...")
        shapefile_name_to_load = shp_files[0]  # Load the first .shp file found
        shape_file_path = os.path.join(output_dir, shapefile_name_to_load)
        try:
            gdf = gpd.read_file(shape_file_path)
            print(f"Shapefile '{shapefile_name_to_load}' loaded successfully.")
            return gdf
        except Exception as e:
            print(f"Error loading shapefile '{shapefile_name_to_load}': {e}")
            return None
    else:
        # If no .shp files are found, attempt to unzip
        print(f"No .shp files found in '{output_dir}'. Attempting to unzip '{zip_filename}'...")
        try:
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)
            print(f"Zip file extracted to '{output_dir}'.")

            # After unzipping, look for .shp files again
            shp_files_after_unzip = [f for f in os.listdir(output_dir) if f.endswith('.shp')]
            if shp_files_after_unzip:
                print(f"Shapefile(s) found after unzipping. Loading the first one...")
                shapefile_name_to_load = shp_files_after_unzip[0]
                shape_file_path = os.path.join(output_dir, shapefile_name_to_load)
                try:
                    gdf = gpd.read_file(shape_file_path)
                    print(f"Shapefile '{shapefile_name_to_load}' loaded successfully.")
                    return gdf
                except Exception as e:
                    print(f"Error loading shapefile '{shapefile_name_to_load}': {e}")
                    return None
            else:
                print(f"No .shp files found in '{output_dir}' after unzipping.")
                return None

        except zipfile.BadZipFile as e:
            print(f"Error unzipping file (Bad zip file): {e}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred during unzipping or loading: {e}")
            return None



# Build request to fetch keyword plan metrics by location
def get_keyword_estimates(client, keywords, geo_ids, batch_size=20):
    """
    Builds a keyword
    """
    keyword_plan_service = client.get_service("KeywordPlanIdeaService")

    results = []

    for state, geo_id in geo_ids.items():
        for i in range(0, len(keywords), batch_size):
            batch_keywords = keywords[i:i + batch_size]

            request = {
                "customer_id": client.login_customer_id,
                "language": "languageConstants/1000",  # English
                "geo_target_constants": [f"geoTargetConstants/{geo_id}"],
                "keyword_plan_network": client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH_AND_PARTNERS,
                "keyword_seed": None,
            }

            try:
                response = keyword_plan_service.generate_keyword_ideas(request=request)

                for result in response:
                    results.append({
                        'state': state,
                        'keyword': result.text,
                        'avg_monthly_searches': result.keyword_idea_metrics.avg_monthly_searches,
                        'competition': result.keyword_idea_metrics.competition.name,
                    })
            except Exception as e:
                print(f"Error in state {state} with geo ID {geo_id}, batch starting with '{batch_keywords[0]}': {e}")

    return pd.DataFrame(results)

import time
import pandas as pd
from google.api_core.exceptions import ResourceExhausted

def get_exact_keyword_volumes(client, keywords, geo_ids, batch_size=20, max_retries=5):
    keyword_plan_service = client.get_service("KeywordPlanIdeaService")
    results = []

    for state, geo_id in geo_ids.items():
        for i in range(0, len(keywords), batch_size):
            batch_keywords = keywords[i:i + batch_size]

            request = {
                "customer_id": client.login_customer_id,
                "language": "languageConstants/1000",
                "geo_target_constants": [f"geoTargetConstants/{geo_id}"],
                "keyword_plan_network": client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH_AND_PARTNERS,
                "keywords": batch_keywords,
                "historical_metrics_options": {
                    "include_average_cpc": False
                },
            }

            retries = 0
            while retries < max_retries:
                try:
                    response = keyword_plan_service.generate_keyword_historical_metrics(request=request)
                    for result in response.results:
                        results.append({
                            'state': state,
                            'keyword': result.text,
                            'avg_monthly_searches': result.keyword_metrics.avg_monthly_searches,
                            'competition': result.keyword_metrics.competition.name,
                        })
                    break  # Exit loop after success
                except ResourceExhausted as e:
                    retry_delay = 2 ** retries
                    print(f"[{state}] Quota exceeded. Retry in {retry_delay} seconds. Attempt {retries+1}/{max_retries}.")
                    time.sleep(retry_delay)
                    retries += 1
                except Exception as e:
                    print(f"[{state}] Error for batch '{batch_keywords[0]}': {e}")
                    break  # Exit loop if non-quota error encountered

            if retries == max_retries:
                print(f"[{state}] Max retries exceeded for batch '{batch_keywords[0]}'. Moving to next batch.")

    return pd.DataFrame(results)
def get_repair_keyword_ideas(client, seed_keywords, geo_id='2840', language_id='1000', suggestions_per_keyword=10, retries=5):
    """
    Fetch keyword ideas per seed keyword, salvaging results if API limit reached.

    Args:
        client: GoogleAdsClient instance.
        seed_keywords (list): Initial seed keywords.
        geo_id (str): Geo target ID ('2840' for US).
        language_id (str): Language ID ('1000' for English).
        suggestions_per_keyword (int): Top suggestions per seed keyword.
        retries (int): Retries on API rate-limit errors.

    Returns:
        pd.DataFrame: Collected keyword suggestions and metrics.
    """
    keyword_plan_service = client.get_service("KeywordPlanIdeaService")
    results = []

    for seed_keyword in seed_keywords:
        request = {
            "customer_id": client.login_customer_id,
            "language": f"languageConstants/{language_id}",
            "geo_target_constants": [f"geoTargetConstants/{geo_id}"],
            "keyword_plan_network": client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH_AND_PARTNERS,
            "keyword_seed": {"keywords": [seed_keyword]},
            "page_size": suggestions_per_keyword
        }

        attempt = 0
        wait_time = 5  # Initial backoff in seconds

        while attempt <= retries:
            try:
                response = keyword_plan_service.generate_keyword_ideas(request=request)

                for result in response:
                    results.append({
                        'seed_keyword': seed_keyword,
                        'suggested_keyword': result.text,
                        'avg_monthly_searches': result.keyword_idea_metrics.avg_monthly_searches,
                        'competition': result.keyword_idea_metrics.competition.name,
                    })
                # Successfully retrieved results; break retry loop
                break

            except ResourceExhausted:
                print(f"Rate limit hit fetching '{seed_keyword}'. Attempt {attempt + 1}/{retries}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
                wait_time *= 2  # Exponential backoff
                attempt += 1

        if attempt > retries:
            print(f"Exceeded max retries for '{seed_keyword}'. Moving to next keyword.")

    # Return all successfully collected results even if some requests failed
    return pd.DataFrame(results)

def prep_paid_search_data(df, all_regions, geo_col='state', neutral_factor=0.75):
    df = df.groupby(geo_col)['avg_monthly_searches'].sum().reset_index()
    df[geo_col] = df[geo_col].str.upper()

    total_searches = df['avg_monthly_searches'].sum()
    if total_searches == 0:
        df['paid_search_composite_factor_100'] = neutral_factor
    else:
        df['paid_search_composite_factor_100'] = (df['avg_monthly_searches'] / total_searches) * 100

    df.loc[df['avg_monthly_searches'] == 0, 'paid_search_composite_factor_100'] = neutral_factor

    # Add missing regions explicitly with neutral factor
    missing_regions = set(all_regions) - set(df[geo_col])
    if missing_regions:
        missing_df = pd.DataFrame({
            geo_col: list(missing_regions),
            'avg_monthly_searches': 0,
            'paid_search_composite_factor_100': neutral_factor
        })
        df = pd.concat([df, missing_df], ignore_index=True)

    return df[[geo_col, 'avg_monthly_searches', 'paid_search_composite_factor_100']].reset_index(drop=True)

def get_trends_via_scrapingbee(
        keyword_list, scrapingbee_api_key, language='en-US', tz=360, geo='US',
        resolution='REGION', timeframe='today 12-m', inc_low_vol=True,
        max_retries=4, initial_wait=10, timeout=(10, 30)):  # increased timeouts clearly

    proxy_url = f"http://{scrapingbee_api_key}:@proxy.scrapingbee.com:8886"
    proxies = [proxy_url]
    dfs = []

    for kw in keyword_list:
        retries = 0
        wait_time = initial_wait

        while retries <= max_retries:
            try:
                pytrends = TrendReq(
                    hl=language,
                    tz=tz,
                    proxies=proxies,
                    timeout=timeout,
                    requests_args={'verify': False}
                )

                pytrends.build_payload([kw], geo=geo, timeframe=timeframe)
                df_kw = pytrends.interest_by_region(resolution=resolution, inc_low_vol=inc_low_vol)
                df_kw.rename(columns={kw: kw.replace(' ', '_')}, inplace=True)
                dfs.append(df_kw)
                print(f"[Success] '{kw}' retrieved successfully.")
                break

            except (RequestException, ReadTimeout) as e:
                error_str = str(e).lower()
                if '429' in error_str or 'too many requests' in error_str or 'timeout' in error_str:
                    print(f"[Retryable Error] '{kw}' - Retrying in {wait_time}s (attempt {retries+1}/{max_retries}): {e}")
                    jitter = random.uniform(2, 5)
                    time.sleep(wait_time + jitter)
                    wait_time *= 2
                    retries += 1
                else:
                    print(f"[Critical Error] '{kw}' - Non-retryable error: {e}")
                    raise

        if retries > max_retries:
            print(f"[Skipped] Exceeded maximum retries for keyword '{kw}'.")

        sleep_between_keywords = random.uniform(5, 10)
        print(f"Waiting {sleep_between_keywords:.1f}s before next keyword...")
        time.sleep(sleep_between_keywords)

    if dfs:
        df_combo = pd.concat(dfs, axis=1)
        print("Data successfully retrieved for some/all keywords.")
    else:
        df_combo = pd.DataFrame()
        print("No data retrieved for keywords after retries.")

    return df_combo

import time
import random
from pytrends.request import TrendReq
from requests.exceptions import RequestException
import pandas as pd

from pytrends.exceptions import TooManyRequestsError

def get_trends(keyword_list, language='en-US', tz=360, geo='US', resolution='REGION',
               timeframe='today 12-m', inc_low_vol=True, max_retries=5, initial_wait=10):

    dfs = []

    for kw in keyword_list:
        retries = 0
        wait_time = initial_wait

        while retries <= max_retries:
            try:
                pytrends = TrendReq(hl=language, tz=tz)
                pytrends.build_payload([kw], geo=geo, timeframe=timeframe)
                df_kw = pytrends.interest_by_region(resolution=resolution, inc_low_vol=inc_low_vol)
                df_kw.rename(columns={kw: kw.replace(' ', '_')}, inplace=True)
                dfs.append(df_kw)
                print(f"Successfully retrieved data for keyword '{kw}'")
                break

            except TooManyRequestsError as e:
                jitter = random.uniform(1, 5)
                print(f"429 TooManyRequestsError for '{kw}', retrying in {wait_time+jitter:.1f}s "
                      f"(attempt {retries+1}/{max_retries})...")
                time.sleep(wait_time + jitter)
                wait_time *= 2
                retries += 1

            except Exception as e:
                print(f"Non-retryable error: {e}")
                raise

        if retries > max_retries:
            print(f"Exceeded maximum retries for keyword '{kw}'. Skipping keyword.")

        sleep_duration = random.uniform(5, 10)
        print(f"Pausing for {sleep_duration:.1f}s before next keyword...")
        time.sleep(sleep_duration)

    if dfs:
        df_combo = pd.concat(dfs, axis=1)
    else:
        df_combo = pd.DataFrame()
        print("Warning: No keyword data successfully retrieved.")

    return df_combo

def prep_trends_data(df, all_regions, region_col='region', neutral_factor=0.75):
    df[region_col] = df[region_col].str.upper().apply(normalize_text)

    numeric_cols = df.select_dtypes(include='number').columns
    df['st_composite_sum'] = df[numeric_cols].sum(axis=1)

    total_sum = df['st_composite_sum'].sum()
    if total_sum == 0:
        df['st_composite_factor_100'] = neutral_factor
    else:
        df['st_composite_factor_100'] = (df['st_composite_sum'] / total_sum) * 100

    df['st_composite_factor_100'] = df['st_composite_factor_100'].clip(upper=5)
    df.loc[df['st_composite_sum'] == 0, 'st_composite_factor_100'] = neutral_factor

    # Add missing regions explicitly with neutral factor
    missing_regions = set(all_regions) - set(df[region_col])
    if missing_regions:
        missing_df = pd.DataFrame({
            region_col: list(missing_regions),
            'st_composite_sum': 0,
            'st_composite_factor_100': neutral_factor
        })
        df = pd.concat([df[[region_col, 'st_composite_sum', 'st_composite_factor_100']], missing_df], ignore_index=True)

    return df[['region', 'st_composite_sum', 'st_composite_factor_100']].reset_index(drop=True)

def add_us_stats_and_geos(gdf, df_trends, df_stats, paid_df, gdf_geo_col='NAME', trends_geo_col='region', stats_geo_col='STATE_NAME', paid_geo_col='state', counts_col=None):
    if counts_col is None:
        return "Error: counts_col must be provided."
    gdf[gdf_geo_col] = gdf[gdf_geo_col].apply(normalize_text)
    df_trends[trends_geo_col] = df_trends[trends_geo_col].apply(normalize_text)
    df_stats[stats_geo_col] = df_stats[stats_geo_col].apply(normalize_text)
    paid_df[paid_geo_col] = paid_df[paid_geo_col].apply(normalize_text)
    merged_df = gdf.merge(df_trends, left_on=gdf_geo_col, right_on=trends_geo_col, how='left')
    merged_df = pd.merge(merged_df, df_stats, left_on=gdf_geo_col, right_on=stats_geo_col, how='left', suffixes=('', '_extra'))
    for col in merged_df.columns:
        if col.endswith('_extra'):
            merged_df.drop(columns=[col], inplace=True)

    merged_df['Ops_below_250k'] = merged_df['Ops_below_250k'].fillna(0).round(0).astype(int)
    merged_df['Ops_250k_or_more'] = merged_df['Ops_250k_or_more'].fillna(0).round(0).astype(int)
    merged_df['Total_Ops'] = merged_df['Total_Ops'].fillna(0).round(0).astype(int)
    comp_col_label = f"{counts_col}_composite_factor_100"
    merged_df[comp_col_label] = (merged_df[counts_col] / merged_df[counts_col].sum()) * 100
    # merged_df['Ops_below_250k_composite_factor_100'] = (merged_df['Ops_below_250k'] / merged_df['Ops_250k_or_more'].sum()) * 100
    merged_df2 = merged_df.merge(paid_df, left_on=gdf_geo_col, right_on=paid_geo_col, how='left').copy().sort_values(gdf_geo_col).reset_index(drop=True)
    # merged_df2 = merged_df2.loc[~merged_df2[paid_geo_col].isna()].copy().sort_values(gdf_geo_col).reset_index(drop=True)


    mean_trends = merged_df2['st_composite_factor_100'].fillna(0).mean()
    mean_volume = merged_df2['paid_search_composite_factor_100'].fillna(0).mean()

    # Clearly calculate relative positions
    merged_df2['trends_relative'] = merged_df2['st_composite_factor_100'] / mean_trends
    merged_df2['volume_relative'] = merged_df2['paid_search_composite_factor_100'] / mean_volume

    # Clearly combine both into single adjustment factor
    merged_df2['combined_relative_factor'] = (merged_df2['trends_relative'] + merged_df2['volume_relative']) / 2

    # Adjusted audience clearly calculated
    audience_label = f"adjusted_audience_{counts_col}"
    merged_df2[audience_label] = (merged_df2[counts_col] * merged_df2['combined_relative_factor']).fillna(0).round(0).astype(int)
    return merged_df2


def reposition_alaska_hawaii(gdf):
    # Separate states
    contiguous_us = gdf[~gdf['NAME'].isin(['ALASKA', 'HAWAII'])]
    alaska = gdf[gdf['NAME'] == 'ALASKA'].copy()
    hawaii = gdf[gdf['NAME'] == 'HAWAII'].copy()
    # Adjusted scale and translate for Alaska and Hawaii
    alaska.geometry = alaska.scale(xfact=0.4, yfact=0.4, origin='center').translate(xoff=1500000, yoff=-4400000)
    hawaii.geometry = hawaii.scale(xfact=0.7, yfact=0.7, origin='center').translate(xoff=5200000, yoff=-1700000)
    # Combine adjusted geometries
    repositioned_us = pd.concat([contiguous_us, alaska, hawaii])
    # and remove all other islands like puerto rico
    repositioned_us = repositioned_us[~(repositioned_us['STATEFP'] >= '60')].sort_values('NAME').reset_index(drop=True)
    return repositioned_us

def plot_us_map(gdf, _density_cmap, _column, _legend_kwds, _title, _footer, _vmax=50000, projection=None, filename='map_output.png', dpi=300):
    if projection is None:
        fig, ax = plt.subplots(figsize=(10, 5))
    else:
        fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': projection})

    gdf.plot(ax=ax, column=_column, cmap=_density_cmap, legend=True, vmin=0, vmax=_vmax,
             legend_kwds=_legend_kwds)

    ax.axis('off')
    ax.set_title(_title, fontsize=14, pad=20, horizontalalignment='center')

    colorbar = ax.get_figure().axes[-1]
    colorbar.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))

    fig.subplots_adjust(bottom=0.1)
    fig.text(0.5, 0.01, _footer, ha='center', fontsize=10)

    # Save figure at higher resolution
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def fetch_google_ads_geo_targets(client, country_codes=['US', 'CA']):
    ga_service = client.get_service("GoogleAdsService")

    country_codes_formatted = ', '.join(f"'{code}'" for code in country_codes)

    query = f"""
        SELECT
            geo_target_constant.resource_name,
            geo_target_constant.id,
            geo_target_constant.name,
            geo_target_constant.country_code,
            geo_target_constant.target_type
        FROM geo_target_constant
        WHERE geo_target_constant.country_code IN ({country_codes_formatted})
        AND geo_target_constant.target_type IN ('State', 'Province', 'Country')
    """

    response = ga_service.search_stream(customer_id=client.login_customer_id, query=query)

    geo_targets = []
    for batch in response:
        for row in batch.results:
            geo_targets.append({
                'resource_name': row.geo_target_constant.resource_name,
                'google_id': row.geo_target_constant.id,
                'name': row.geo_target_constant.name,
                'country_code': row.geo_target_constant.country_code,
                'target_type': row.geo_target_constant.target_type,
            })

    return pd.DataFrame(geo_targets)

def load_or_download_csv(local_filename, data_url):
    """
    Loads a CSV file locally or downloads it if not available locally.
    Handles CSV files compressed in ZIP, GZIP, or uncompressed formats.

    Args:
        local_filename (str): Path to the local CSV file.
        data_url (str): URL to download the CSV from if not available locally.

    Returns:
        pd.DataFrame: Loaded dataframe from CSV.
    """
    if os.path.exists(local_filename):
        try:
            df = pd.read_csv(local_filename)
            print(f"Loaded data successfully from {local_filename}.")
            return df
        except Exception as e:
            print(f"Error loading local file, will attempt download: {e}")

    print(f"Downloading data from {data_url}...")
    response = requests.get(data_url, stream=True)
    response.raise_for_status()

    content_type = response.headers.get('Content-Type', '').lower()

    # Handle ZIP compressed files
    if 'zip' in content_type or data_url.lower().endswith('.zip'):
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            csv_files = [name for name in z.namelist() if name.endswith('.csv')]
            if not csv_files:
                raise ValueError("No CSV file found in ZIP archive.")
            with z.open(csv_files[0]) as f:
                df = pd.read_csv(f)

    # Handle GZIP compressed files
    elif 'gzip' in content_type or data_url.lower().endswith('.gz'):
        with gzip.open(BytesIO(response.content), 'rt') as f:
            df = pd.read_csv(f)

    # Handle regular CSV files
    else:
        df = pd.read_csv(BytesIO(response.content))

    # Save locally
    df.to_csv(local_filename, index=False)
    print(f"Data downloaded and saved locally to {local_filename}.")

    return df


# Clearly define Canadian sales categories explicitly
below_250k_categories_canada = [
    '$0',
    '$1 to $9,999',
    '$10,000 to $24,999',
    '$25,000 to $49,999',
    '$50,000 to $99,999',
    '$100,000 to $249,999'
]

above_250k_categories_canada = [
    '$250,000 to $499,999',
    '$500,000 to $999,999',
    '$1,000,000 to $1,999,999',
    '$2,000,000 and over'
]

# Function to categorize revenues explicitly
def categorize_revenues(df, geo_level, geo_col, country_geo_val, above_250k_cats, below_250k_cats, country, pivot_col='Total farm revenues distribution'):
    if country == 'CA':
        df_filtered = df.loc[df[geo_col] != country_geo_val] if geo_level != 'national' else df.loc[df[geo_col] == country_geo_val]
        df_filtered = df[(df[geo_col] != country_geo_val) & ~(df[geo_col].str.contains(','))] if geo_level == 'provincial' else df_filtered
        # Example assuming your dataframe is df:
        summary_df = df_filtered.pivot_table(
            index=geo_col,
            columns=pivot_col,
            values='VALUE',
            aggfunc='sum',
            fill_value=0
        ).reset_index()

        # Explicit aggregation for below and above $250k
        summary_df['Ops_below_250k'] = summary_df[below_250k_cats].sum(axis=1)
        summary_df['Ops_250k_or_more'] = summary_df[above_250k_cats].sum(axis=1)
        summary_df['geo_code'] = summary_df[geo_col].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0])
        summary_df['geo_name'] = summary_df[geo_col].apply(lambda x: x.split('[')[0].strip()).str.upper()

        summary_df['Total_Ops'] = summary_df['Ops_below_250k'] + summary_df['Ops_250k_or_more']
        summary_df['PRUID'] = (
            summary_df['geo_code']
            .str.replace('PR', '', regex=False)  # clearly remove 'PR' if present
            .str[:2]                             # clearly take the first two characters
            .astype(int)                         # clearly convert to integer
        )
        final_columns = ['PRUID','geo_name', 'geo_code', 'Ops_below_250k', 'Ops_250k_or_more', 'Total_Ops']
        summary_df = summary_df[final_columns]
        summary_df.columns = final_columns
        summary_df = summary_df.copy().sort_values('PRUID').reset_index(drop=True)
        return summary_df[final_columns].reset_index(drop=True)

    else:
        print('not configured for US yet')

def extract_2_vectorized_stats(df, geo_level, geo_col, country_geo_val, pivot1, pivot2, pivot1_label, pivot2_label, pivot_total_label, country, pivot_col, values_col, geo_code_col, geo_name_col, fips_col, agg_function='sum'):
    if country == 'CA':
        df_filtered = df.loc[df[geo_col] != country_geo_val] if geo_level != 'national' else df.loc[df[geo_col] == country_geo_val]
        df_filtered = df[(df[geo_col] != country_geo_val) & ~(df[geo_col].str.contains(','))] if geo_level == 'provincial' else df_filtered
        # Example assuming your dataframe is df:
        summary_df = df_filtered.pivot_table(
            index=geo_col,
            columns=pivot_col,
            values=values_col,
            aggfunc=agg_function,
            fill_value=0
        ).reset_index()

        # Explicit aggregation for below and above $250k
        summary_df[pivot1_label] = summary_df[pivot1].sum(axis=1)
        summary_df[pivot2_label] = summary_df[pivot2].sum(axis=1)
        summary_df[geo_code_col] = summary_df[geo_col].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0])
        summary_df[geo_name_col] = summary_df[geo_col].apply(lambda x: x.split('[')[0].strip()).str.upper()

        summary_df[pivot_total_label] = summary_df[pivot1_label] + summary_df[pivot2_label]
        summary_df[fips_col] = (
            summary_df[geo_code_col]
            .str.replace('PR', '', regex=False)  # clearly remove 'PR' if present
            .str[:2]                             # clearly take the first two characters
            .astype(int)                         # clearly convert to integer
        )
        final_columns = [fips_col, geo_code_col, geo_name_col, pivot1_label, pivot2_label, pivot_total_label]
        summary_df = summary_df[final_columns]
        summary_df.columns = final_columns
        summary_df = summary_df.copy().sort_values(fips_col).reset_index(drop=True)
        return summary_df[final_columns].reset_index(drop=True)

    else:
        print('not configured for US yet')

def load_or_download_csv(local_filename, data_url):
    """
    Loads a CSV file locally or downloads it if not available locally.
    Handles CSV files compressed in ZIP, GZIP, or uncompressed formats.

    Args:
        local_filename (str): Path to the local CSV file.
        data_url (str): URL to download the CSV from if not available locally.

    Returns:
        pd.DataFrame: Loaded dataframe from CSV.
    """
    if os.path.exists(local_filename):
        try:
            df = pd.read_csv(local_filename)
            print(f"Loaded data successfully from {local_filename}.")
            return df
        except Exception as e:
            print(f"Error loading local file, will attempt download: {e}")

    print(f"Downloading data from {data_url}...")
    response = requests.get(data_url, stream=True)
    response.raise_for_status()

    content_type = response.headers.get('Content-Type', '').lower()

    # Handle ZIP compressed files
    if 'zip' in content_type or data_url.lower().endswith('.zip'):
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            csv_files = [name for name in z.namelist() if name.endswith('.csv')]
            if not csv_files:
                raise ValueError("No CSV file found in ZIP archive.")
            with z.open(csv_files[0]) as f:
                df = pd.read_csv(f)

    # Handle GZIP compressed files
    elif 'gzip' in content_type or data_url.lower().endswith('.gz'):
        with gzip.open(BytesIO(response.content), 'rt') as f:
            df = pd.read_csv(f)

    # Handle regular CSV files
    else:
        df = pd.read_csv(BytesIO(response.content))

    # Save locally
    df.to_csv(local_filename, index=False)
    print(f"Data downloaded and saved locally to {local_filename}.")

    return df

def simplify_geom(geom, tolerance=0.1):
    return geom.simplify(tolerance, preserve_topology=True)

def safe_extract(x, item='value'):
    try:
        val = x[0][item]
        return val.strip() if isinstance(val, str) else val
    except (IndexError, KeyError, TypeError, AttributeError):
        return None


def load_trends_from_search_api(kw_list, api_key, geo='US'):
    dfs = []
    kw_list = kw_list if isinstance(kw_list, list) else [kw_list]
    url = f'https://www.searchapi.io/api/v1/search?api_key={api_key}'
    for kw in kw_list:
        params = {
            "engine": "google_trends",
            "q": kw,
            "data_type": "GEO_MAP",
            "tz": 360,
            "geo": geo,
            "resolution": "REGION",
            "timeframe": "today 12-m",
            "inc_low_vol": True,
            "region": "REGION",
            "api_key": api_key
        }

        response = requests.get(url, params=params)
        interest_raw = response.json().get('interest_by_region', [])

        df = pd.DataFrame(interest_raw)
        if df.empty:
            print(f"No data found for keyword: {kw}")
            continue
        df['region'] = df['name'].str.strip().str.upper()

        val_col_name = f"{kw.replace(' ', '_').lower()}"
        df[val_col_name] = df['values'].apply(safe_extract, item='extracted_value')
        df[val_col_name] = df[val_col_name].fillna(0).astype(int)

        dfs.append(df[['region', val_col_name]])

    # Concatenate and explicitly aggregate by region
    combo_df = pd.concat(dfs)
    combo_df = combo_df.groupby('region').sum()

    return combo_df

import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def safe_extract(value, item='extracted_value'):
    if isinstance(value, dict):
        return value.get(item, 0)
    return 0

def fetch_trend_for_kw(kw, api_key, geo='US'):
    url = f'https://www.searchapi.io/api/v1/search'
    params = {
        "engine": "google_trends",
        "q": kw,
        "data_type": "GEO_MAP",
        "tz": 360,
        "geo": geo,
        "resolution": "REGION",
        "timeframe": "today 12-m",
        "inc_low_vol": True,
        "region": "REGION",
        "api_key": api_key
    }

    response = requests.get(url, params=params)
    interest_raw = response.json().get('interest_by_region', [])

    df = pd.DataFrame(interest_raw)
    if df.empty:
        print(f"No data found for keyword: {kw}")
        return None

    df['region'] = df['name'].str.strip().str.upper()

    val_col_name = f"{kw.replace(' ', '_').lower()}"
    df[val_col_name] = df['values'].apply(safe_extract, item='extracted_value')
    df[val_col_name] = df[val_col_name].fillna(0).astype(int)

    return df[['region', val_col_name]]

def load_trends_from_search_api_parallel(kw_list, api_key, geo='US', max_threads=10):
    kw_list = kw_list if isinstance(kw_list, list) else [kw_list]

    dfs = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        future_to_kw = {
            executor.submit(fetch_trend_for_kw, kw, api_key, geo): kw for kw in kw_list
        }

        for future in as_completed(future_to_kw):
            kw = future_to_kw[future]
            try:
                df = future.result()
                if df is not None:
                    dfs.append(df)
            except Exception as e:
                print(f"Exception for keyword {kw}: {e}")

    # Concatenate and aggregate
    if dfs:
        combo_df = pd.concat(dfs)
        combo_df = combo_df.groupby('region').sum()
        return combo_df
    else:
        return pd.DataFrame()

def estimate_audience_distribution(search_df, population_df, total_audience=38_000_000):
    # Sum total searches per state
    state_searches = search_df.groupby(['geo_name'])['avg_monthly_searches'].sum().reset_index()

    # Merge with population data
    merged_df = state_searches.merge(population_df, on='geo_name', how='inner')

    # Calculate Search-to-Population Index (SPI)
    merged_df['SPI'] = merged_df['avg_monthly_searches'] / merged_df['total_population']

    # Normalize SPI
    merged_df['normalized_SPI'] = merged_df['SPI'] / merged_df['SPI'].sum()

    # Estimate audience per state
    merged_df['estimated_audience'] = (merged_df['normalized_SPI'] * total_audience).astype(int)

    return merged_df[['geo_code', 'geo_name', 'total_population', 'avg_monthly_searches', 'estimated_audience']]


def estimate_search_population_indexes(search_df, population_df, population_col='total_population', baseline_audince_factor=None, interest_multiplier=None):
    # Sum total searches per state
    state_searches = search_df.groupby(['geo_name'])['avg_monthly_searches'].sum().reset_index()

    # Merge with population data
    merged_df = state_searches.merge(population_df, on='geo_name', how='inner')


    # Calculate Search-to-Population Index (SPI)
    merged_df['SPI'] = merged_df['avg_monthly_searches'] / merged_df[population_col]

    # Normalize SPI
    merged_df['normalized_SPI'] = merged_df['SPI'] / merged_df['SPI'].sum()
    median_normalized_SPI = merged_df['normalized_SPI'].median()
    merged_df['median_normalized_SPI'] = median_normalized_SPI
    merged_df['interest_scaling_factor'] = merged_df['normalized_SPI'] - merged_df['median_normalized_SPI']
    if baseline_audince_factor is not None and interest_multiplier is not None:
        merged_df['estimated_audience_factor'] = (baseline_audince_factor + (merged_df['interest_scaling_factor']))
    else:
        merged_df['estimated_audience_factor'] = np.nan
    return merged_df[['geo_code', 'geo_name', population_col, 'avg_monthly_searches', 'SPI', 'normalized_SPI', 'median_normalized_SPI', 'interest_scaling_factor', 'estimated_audience_factor']]

def estimate_regional_search_interest(search_df, population_df):
    # Sum total searches per state
    state_searches = search_df.groupby(['geo_name'])['avg_monthly_searches'].sum().reset_index()
    total_searches = state_searches['avg_monthly_searches'].sum()
    state_searches['avg_monthly_interest_percent'] = state_searches['avg_monthly_searches'] / total_searches

    # Merge with population data
    merged_df = state_searches.merge(population_df, on='geo_name', how='inner')

    # Calculate Search-to-Population Index (SPI)
    merged_df['SPI'] = merged_df['avg_monthly_searches'] / merged_df['total_population']

    # Normalize SPI
    merged_df['normalized_SPI'] = merged_df['SPI'] / merged_df['SPI'].sum()

    return merged_df[['geo_code', 'geo_name', 'total_population', 'avg_monthly_searches', 'SPI', 'normalized_SPI']]


def convert_state_abbrev_to_full(df, abbrev_col='state'):
    df['state_full'] = df[abbrev_col].apply(lambda x: us.states.lookup(x).name if us.states.lookup(x) else x).str.upper()
    df['geo_name'] = df['state_full'].str.upper()
    df['geo_code'] = df[abbrev_col]
    return df

In [ ]:
# @title Load US and Canada State / Province shapefiles

# Remove GDAL's GeoJSON size limit explicitly
os.environ['OGR_GEOJSON_MAX_OBJ_SIZE'] = '0'

try:
    gdf_us_states = gpd.read_file('simplified_us_state_geos.gpkg')
except Exception as e:
    print("File not found. Attempting to fetch and load shapefile.")

    geo_address = 'https://www2.census.gov/geo/tiger/TIGER2023/STATE/tl_2023_us_state.zip'

    gdf_us_states = (fetch_unzip_load_shapefile_flexible(geo_address, 'state_tiger').to_crs('EPSG:5070'))
# gdf_us_states = gdf_us_states.simplify(
#         tolerance=0.05,  # Adjust this if needed; higher = more simplified
#         preserve_topology=True
#     )
# gdf_us_states = gdf_us_states.simplify()

# https://www2.census.gov/geo/tiger/TIGER2024/ESTATE/tl_2024_78_estate.zip

# Load geographic shapes for US/Canada
# gdf_us_states = gpd.read_file('/content/us_state/tl_2024_us_state.shp').to_crs('EPSG:5070')
    gdf_us_states_ref = gdf_us_states[['STATEFP', 'NAME']].drop_duplicates().sort_values('STATEFP').reset_index(drop=True)
    gdf_us_states_ref['NAME'] = gdf_us_states_ref['NAME'].str.upper()
    gdf_us_states['NAME'] = gdf_us_states['NAME'].str.upper()

# save this ref to a gcs bucket
    gdf_us_states_ref.to_csv('us_state_ref.csv', index=False)
    with mp.Pool(mp.cpu_count()) as pool:
        simplified_geometries = pool.map(simplify_geom, gdf_us_states.geometry)
    gdf_us_states['geometry'] = simplified_geometries
    gdf_us_states.to_file('simplified_us_state_geos.gpkg', driver='GPKG')



repositioned_us = reposition_alaska_hawaii(gdf_us_states).sort_values('STATEFP').reset_index(drop=True)

# Example usage:


try:
    canada_provinces_gdf = gpd.read_file('simplified_canada_province_geos.gpkg')
except Exception as e:
    print("File not found. Attempting to fetch and load shapefile.")
    url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lpr_000b21a_e.zip?st=wk4IrLBG"
    output_directory = '/content/province-shapes/'
    canada_provinces_gdf = fetch_unzip_load_shapefile_flexible(url, output_directory)
# canada_provinces_gdf = canada_provinces_gdf.simplify(
#         tolerance=0.05,  # Adjust this if needed; higher = more simplified
#         preserve_topology=True
#     )

    if canada_provinces_gdf is not None:
        canada_provinces_gdf['PRENAME'] = canada_provinces_gdf['PRENAME'].str.upper()
        canada_provinces_gdf = canada_provinces_gdf.to_crs('EPSG:5070')
        print("\nGeoDataFrame loaded:")
        print(canada_provinces_gdf.head())
        # # Adjust the tolerance to balance simplification and detail retention
        # simplification_tolerance = 0.05  # Example tolerance (increase to simplify more aggressively)
        # # Simplify geometries explicitly
        # canada_provinces_gdf['geometry'] = canada_provinces_gdf['geometry'].simplify(
        #     tolerance=simplification_tolerance, preserve_topology=True
        # )
        # # Ensure geometries remain valid
        # canada_provinces_gdf = canada_provinces_gdf[canada_provinces_gdf.is_valid]
        with mp.Pool(mp.cpu_count()) as pool:
            simplified_geometries = pool.map(simplify_geom, canada_provinces_gdf.geometry)

        canada_provinces_gdf['geometry'] = simplified_geometries
        canada_provinces_gdf = canada_provinces_gdf.copy().sort_values('PRUID').reset_index(drop=True)
        canada_provinces_gdf.to_file('simplified_canada_province_geos.gpkg', driver='GPKG')
    else:
        print("No valid GeoDataFrame loaded.")




In [ ]:
#  @title farm operator repair keywords
us_high_revenue_final_keywords = [
    "tractor hydraulic repair",
    "diesel tractor troubleshooting",
    "tractor transmission repair",
    "combine maintenance",
    "farm equipment diagnostics",
    "precision agriculture calibration",
    "tractor emissions repair",
    "planter upgrades",
    "tractor GPS installation",
    "tractor AC repair",
    "OEM tractor parts",
    "tractor performance tuning",
    "tractor PTO troubleshooting",
    "tractor repair manuals",
    "tractor ECU issues",
    "yield monitor setup",
    "hydraulic retrofitting",
    "tractor service training",
    "preventive tractor maintenance",
    "engine rebuild for tractors",
    "John Deere equipment maintenance",
    "John Deere diagnostic tools",
    "john deere part",
    "tractor winter preparation",
    "tractor block heater install",
    "tractor engine overhaul",
    "tractor electrical diagnostics",
    "tractor hydraulic systems",
    "tractor engine upgrades",
    "tractor troubleshooting guide",
    "tractor attachment repair",
]

ca_high_revenue_final_keywords = [
    "tractor repair Canada",
    "diesel tractor service",
    "combine maintenance",
    "tractor diagnostics tools",
    "precision agriculture Canada",
    "tractor GPS systems",
    "tractor AC service",
    "tractor emission repair",
    "large tractor tires",
    "farm planter equipment",
    "OEM tractor parts Canada",
    "tractor upgrades Canada",
    "tractor PTO repair",
    "tractor repair manuals",
    "tractor preventive maintenance",
    "tractor training courses",
    "hydraulic systems repair",
    "tractor engine rebuild",
    "tractor winterization Canada",
    "tractor block heater",
    "farm equipment repair",
    "agricultural machinery repair",
    "tractor fuel system service",
    "farm machinery maintenance",
    "equipment hydraulic repair",
    "agricultural machinery parts",
    "heavy equipment repair",
    "farm equipment parts",
    "tractor servicing",
    "tractor electronic diagnostics",
    # French-language keywords
    "réparation tracteur agricole",
    "entretien diesel tracteur",
    "diagnostic tracteur",
    "manuel entretien tracteur",
    "chauffe-bloc tracteur",
    "hivernisation tracteur",
    "pièces équipement agricole",
    "réparation équipement agricole",
]

us_low_revenue_final_keywords = [
    "tractor oil change",
    "lawn mower repair",
    "compact tractor issues",
    "small tractor maintenance",
    "tractor belt replacement",
    "tractor battery issues",
    "tractor tire replacement",
    "tractor decal restoration",
    "antique tractor refurbishing",
    "tractor painting guide",
    "tractor seat repair",
    "tractor hydraulic leak fix",
    "tractor rust removal",
    "tractor basic maintenance",
    "john deere part",
    "tractor DIY videos",
    "tractor aftermarket parts",
    "tractor carburetor repair",
    "garden tractor upkeep",
    "tractor electrical issues",
    "John Deere lawn tractor repair",
    "compact tractor attachments",
    "tractor winterizing",
    "tractor snowblower upkeep",
    "cold weather tractor battery",
    "tractor engine tune-up",
    "tractor starter repair",
    "tractor maintenance schedule",
    "tractor troubleshooting",
    "tractor attachments DIY",
    "tractor parts online",
]

ca_low_revenue_final_keywords = [
    "tractor oil change Canada",
    "compact tractor problems",
    "small tractor repair",
    "tractor belt replacement",
    "tractor battery replacement",
    "compact tractor tires",
    "tractor restoration",
    "tractor upholstery",
    "tractor hydraulic leaks",
    "tractor rust removal",
    "tractor carburetor repair",
    "tractor maintenance Canada",
    "tractor parts Canada",
    "aftermarket tractor parts",
    "tractor electrical repair",
    "tractor DIY",
    "tractor snowblower maintenance",
    "tractor block heater",
    "tractor winter preparation",
    "garden tractor repair",
    "compact tractor care",
    "farm tractor repair",
    "lawn tractor maintenance",
    "small engine tractor repair",
    "agricultural equipment maintenance",
    "tractor troubleshooting",
    "equipment parts Canada",
    "tractor attachment repair",
    # French-language keywords
    "vidange huile tracteur",
    "entretien tracteur",
    "réparation tondeuse",
    "chauffe-bloc tracteur",
    "hivernisation tracteur",
    "entretien souffleuse",
    "réparation rouille tracteur",
    "pièces tracteur Canada",
    "manuel réparation tracteur",
]
standard_diy_indicator_keywords = [
    "John Deere parts",
    "John Deere tractor parts",
    "John Deere repair",
    "John Deere service manual",
    "John Deere tractor maintenance",
    "JD tractor troubleshooting",
    "JD tractor parts online",
    "John Deere oil change",
    "John Deere tractor accessories",
    "John Deere tractor filters",
    "JD parts catalog",
    "John Deere DIY repair",
    "John Deere aftermarket parts",
    "John Deere mower repair",
    "JD tractor belts",
    "John Deere hydraulic repair",
    "John Deere battery replacement",
    "John Deere tractor tires",
    "John Deere tractor attachments",
    "John Deere tractor electrical repair",
    "John Deere DIY parts",
    "John Deere tractor fluid changes",
    "John Deere engine maintenance",
    "John Deere tractor troubleshooting",
    "John Deere tractor DIY videos",
    "John Deere replacement blades",
    "JD tractor lighting kits",
    "John Deere maintenance schedule",
    "JD parts near me",
    "John Deere tractor restoration",
]

In [ ]:
# @title prep state and provinces with population and google location ids for states and provinces

# load us census data

# Convert necessary columns to integers explicitly
age_vars = ['B01001_001E', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E',
            'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E',
            'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E',
            'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E']

acs_variables = {
    "B01003_001E": "total_population",
    "B25001_001E": "total_housing_units",
    "B25003_001E": "total_owner_occupied_units",
    "B01001_003E": "male_under_5",
    "B01001_004E": "male_5_to_9",
    "B01001_005E": "male_10_to_14",
    "B01001_006E": "male_15_to_17",
    "B01001_027E": "female_under_5",
    "B01001_028E": "female_5_to_9",
    "B01001_029E": "female_10_to_14",
    "B01001_030E": "female_15_to_17",
}

df_acs_pop = c.acs5.get(age_vars, {'for': 'county:*'})
df_acs_pop = pd.DataFrame(df_acs_pop)



for col in age_vars:
    df_acs_pop[col] = df_acs_pop[col].astype(int)

# Under 18 calculation
df_acs_pop['under_18'] = (
    df_acs_pop[['B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E',
                  'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E']].sum(axis=1)
)

# 65 and older calculation
df_acs_pop['age_65_plus'] = (
    df_acs_pop[['B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E',
                  'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E']].sum(axis=1)
)

# Adults 18–64 clearly calculated:
df_acs_pop['adults_18_to_64'] = df_acs_pop['B01001_001E'] - (df_acs_pop['under_18'] + df_acs_pop['age_65_plus'])


# for col in acs_variables.values():
#     df_acs_pop[col] = pd.to_numeric(df_acs_pop[col]).astype(int)

# df_acs_pop['under_18'] = (
#     df_acs_pop['male_under_5'].astype(int) + df_acs_pop['female_under_5'].astype(int) +
#     df_acs_pop['male_5_to_9'].astype(int) + df_acs_pop['female_5_to_9'].astype(int) +
#     df_acs_pop['male_10_to_14'].astype(int) + df_acs_pop['female_10_to_14'].astype(int) +
#     df_acs_pop['male_15_to_17'].astype(int) + df_acs_pop['female_15_to_17'].astype(int)
# )

# df_acs_pop['18_and_over'] = df_acs_pop['total_population'] - df_acs_pop['under_18']
# df_acs_pop['total_population_all'] = df_acs_pop['18_and_over']
# df_acs_pop = df_acs_pop.rename(columns={'18_and_over': 'total_population'})

# Convert numeric columns

# Create full FIPS code
df_acs_pop['FIPS'] = df_acs_pop['state'] + df_acs_pop['county']

df_us_state_pop = df_acs_pop[['state', 'adults_18_to_64']].groupby('state').sum().sort_values(by='state').reset_index()


df_us_state_pop_clean = convert_state_abbrev_to_full(df_us_state_pop)
df_us_state_pop_clean['geo_country'] = 'us'
df_us_state_pop_clean['geo_code_na'] = df_us_state_pop_clean['geo_country'] + '-' + df_us_state_pop_clean['geo_code']
df_us_state_pop = df_us_state_pop_clean.loc[((df_us_state_pop_clean['state'] != '11') & (df_us_state_pop_clean['state'].astype(int) < 58)),  ['geo_code','geo_code_na', 'geo_name', 'geo_country', 'adults_18_to_64']].copy().sort_values(by='geo_code').reset_index(drop=True)
del df_us_state_pop_clean
gc.collect()
df_us_state_pop['adults_18_to_64'].sum()
# 202,684,160 adults 18 - 64
# load ca census population data 202,684,160

# ca_census_vars = {
#     "Population and dwelling counts (13): Population, 2021 [1]": "total_population",
#     "Population and dwelling counts (13): Private dwellings occupied by usual residents, 2021 [7]": "total_private_dwellings"
# }
# ca_age_exclude = '0 to 17 years'
ca_pop_fields = {"18 to 64 years": "adults_18_to_64"}
ca_pop_df = sc.table_to_df("17-10-0005-01") # canadian population
recent_ca_pop = ca_pop_df[ca_pop_df['REF_DATE'] == ca_pop_df['REF_DATE'].max()]
# recent_ca_pop.loc[recent_ca_pop['DGUID'].str.len() == 11]


recent_ca_pop['VALUE'] = recent_ca_pop['VALUE'].fillna(0.0).astype(int)
# ca total pop 21,962,032
# ca 18 to 64 12,429,081
recent_ca_pop_18_64 = recent_ca_pop.loc[(recent_ca_pop['Age group'].isin(list(ca_pop_fields.keys()))) & (recent_ca_pop['Gender'] == 'Total - gender')].copy()

# for col in ca_census_vars.keys():
#     ca_pop_df[col] = pd.to_numeric(ca_pop_df[col]).fillna(0.0).astype(int)
# ca_pop_df = ca_pop_df.rename(columns=ca_census_vars)


ca_province_pop_df = recent_ca_pop_18_64[recent_ca_pop_18_64['DGUID'].str.len() == 11].copy().reset_index(drop=True)
ca_province_pop_df['adults_18_to_64'] = ca_province_pop_df['VALUE'].fillna(0.0).astype(int)
ca_province_pop_df['geo_code'] = ca_province_pop_df['DGUID'].str[-2:]
ca_province_pop_df['geo_country'] = 'ca'
ca_province_pop_df['geo_code_na'] = ca_province_pop_df['geo_country'] + '-' +  ca_province_pop_df['DGUID'].str[-2:]
ca_province_pop_df['geo_name'] = ca_province_pop_df['GEO'].str.upper()
ca_province_pop_df = ca_province_pop_df[['geo_code','geo_code_na', 'geo_name', 'geo_country', 'adults_18_to_64']].sort_values(by='geo_code').copy().reset_index(drop=True)
ca_province_pop_df['adults_18_to_64'].sum()
# 25,844,882 adults 18 to 64

# load google geos for states and provinces
google_geos = fetch_google_ads_geo_targets(client)


google_us_states = google_geos[((google_geos['country_code'] == 'US') & (google_geos['target_type'] == 'State'))].copy().sort_values('name').reset_index(drop=True)
google_us_states = google_us_states[['name', 'google_id']].sort_values('name').reset_index(drop=True)
google_us_states['name'] = google_us_states['name'].str.upper()
df_us_state_pop = pd.merge(df_us_state_pop, google_us_states, left_on='geo_name', right_on='name', how='left')

# google_us_states.rename(columns={'Name': 'NAME', 'Criteria ID': 'geo_id'}, inplace=True)
# us_states_google = google_us_states.set_index('name')['id'].to_dict()
google_ca_provinces = google_geos[((google_geos['country_code'] == 'CA') & (google_geos['target_type'] == 'Province'))].copy().sort_values('name').reset_index(drop=True)
google_ca_provinces['name'] = google_ca_provinces['name'].str.upper()
google_ca_provinces = google_ca_provinces[['name', 'google_id']].sort_values('name').reset_index(drop=True)
ca_province_pop_df = pd.merge(ca_province_pop_df, google_ca_provinces, left_on='geo_name', right_on='name', how='left')
ca_province_pop_df['google_id'] = ca_province_pop_df['google_id'].fillna(0.0).astype(int)
# ca_province_pop_df.rename(columns={'Name': 'NAME', 'Criteria ID': 'geo_id'}, inplace=True
# ca_provinces = google_ca_provinces.set_index('name')['id'].to_dict()
merge_cols = ['geo_country', 'geo_code','geo_code_na', 'geo_name', 'google_id',  'adults_18_to_64']
na_population_by_region = pd.concat([df_us_state_pop[merge_cols], ca_province_pop_df[merge_cols]], ignore_index=True).copy()

us_states_google = df_us_state_pop.set_index('geo_name')['google_id'].to_dict()
ca_provinces_google = ca_province_pop_df.loc[ca_province_pop_df['google_id'] != 0].set_index('geo_name')['google_id'].to_dict()
na_regions_google = {**us_states_google, **ca_provinces_google}


In [ ]:

# kws = us_high_revenue_keywords
kws = us_high_revenue_final_keywords

paid_trends = get_exact_keyword_volumes(client, kws, us_states_google)
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].astype(int)
paid_trends['avg_monthly_searches'].sum()
paid_trends['geo_name'] = paid_trends['state']
us_over250_paid_trends = paid_trends

kws = us_low_revenue_final_keywords


paid_trends = get_exact_keyword_volumes(client, kws, us_states_google)
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].astype(int)
paid_trends['avg_monthly_searches'].sum()
paid_trends['geo_name'] = paid_trends['state']
us_under_250_paid_trends = paid_trends

kws = ca_high_revenue_final_keywords
ca_paid_trends = get_exact_keyword_volumes(client, kws, ca_provinces_google)
ca_paid_trends['avg_monthly_searches'] = ca_paid_trends['avg_monthly_searches'].astype(int)
ca_paid_trends['avg_monthly_searches'].sum()
ca_paid_trends['geo_name'] = ca_paid_trends['state']
ca_over250_paid_trends = ca_paid_trends
ca_over250_paid_trends['avg_monthly_searches'].sum()

kws = ca_low_revenue_final_keywords
ca_paid_trends = get_exact_keyword_volumes(client, kws, ca_provinces_google)
ca_paid_trends['avg_monthly_searches'] = ca_paid_trends['avg_monthly_searches'].astype(int)
ca_paid_trends['avg_monthly_searches'].sum()
ca_paid_trends['geo_name'] = ca_paid_trends['state']
ca_under250_paid_trends = ca_paid_trends

# trt interest

# 96300
# deep_lg_op_diy_results = paid_trends
# deep_lg_op_diy_results['avg_monthly_searches'].sum()
# 1420
# estimate_search_population_indexes




In [ ]:
# @title Load USDA and Canada Agriculture census data
# URL to USDA Census of Agriculture 2022 data (corrected link)
data_url = 'https://www.nass.usda.gov/datasets/qs.census2022.txt.gz'

# Download the GZIP file
local_filename = 'us_agriculture.csv'
try:
    df = pd.read_csv(local_filename)
    print("Data loaded successfully!")
except Exception as e:
    print("Error loading data: - downloading", e)
    response = requests.get(data_url)
    response.raise_for_status()
    # Decompress and load the file into a pandas DataFrame
    with gzip.open(BytesIO(response.content), 'rt') as f:
        df = pd.read_csv(f, delimiter='\t')  # adjust delimiter if necessary
        df.to_csv(local_filename, index=False)
df_us = df.copy()

# data_url = base_url = "https://www150.statcan.gc.ca/n1/en/tbl/csv/32100239-eng.zip?st=wk4IrLBG"
# local_filename = 'ca_agriculture.csv'
# df_canada = load_or_download_csv(local_filename, data_url)
df_canada = sc.table_to_df("32100239")

# df_canada = df_canada.rename(columns={'Total farm revenues distribution': 'revenue_distribution'})
# df_canada['Total farm revenues distribution'].unique()


geo_level='provincial'
provincial_summary_canada = categorize_revenues(df_canada, geo_level=geo_level, geo_col='GEO', country_geo_val='Canada [000000000]', above_250k_cats=above_250k_categories_canada, below_250k_cats=below_250k_categories_canada, country='CA')

print('/nn')
print(provincial_summary_canada.head())



In [ ]:
# workig on maturity model

In [ ]:
['SOURCE_DESC',
 'SECTOR_DESC',
 'GROUP_DESC',
 'COMMODITY_DESC',
 'CLASS_DESC',
 'PRODN_PRACTICE_DESC',
 'UTIL_PRACTICE_DESC',
 'STATISTICCAT_DESC',
 'UNIT_DESC',
 'SHORT_DESC',
 'DOMAIN_DESC',
 'DOMAINCAT_DESC',
 'AGG_LEVEL_DESC',
 'STATE_ANSI',
 'STATE_FIPS_CODE',
 'STATE_ALPHA',
 'STATE_NAME',
 'ASD_CODE',
 'ASD_DESC',
 'COUNTY_ANSI',
 'COUNTY_CODE',
 'COUNTY_NAME',
 'REGION_DESC']

df_us.columns.to_list()

df_us['SECTOR_DESC'].drop_duplicates()
# sector DEMOGRAPHICS
#  grou_desc PRODUCERS, INCOME, FRUIT & TREE NUTS, CROP TOTALS
age_groups = ['AGE GE 75',
 'AGE 25 TO 34',
 'AGE 55 TO 64',
 'AGE 45 TO 54',
 'AGE LT 25',
 'AGE 35 TO 44',
 'AGE 65 TO 74']
df_us.loc[df_us['SECTOR_DESC'] == 'DEMOGRAPHICS', 'GROUP_DESC'].drop_duplicates()
df_us_age = df_us.loc[(df_us['SECTOR_DESC'] == 'DEMOGRAPHICS') & (df_us['GROUP_DESC'] == 'PRODUCERS') & (df_us['UNIT_DESC'] == 'PRODUCERS') & (df_us['SHORT_DESC'].str.contains('AGE') )& (df_us['CLASS_DESC'].isin(age_groups) & (df_us['AGG_LEVEL_DESC'] == 'STATE'))].drop_duplicates().copy().reset_index(drop=True)
df_us_age['VALUE_NUMERIC'] = pd.to_numeric(df_us_age['VALUE'].str.replace(',', ''), errors='coerce')
# df_us_age['COUNTY_CODE'] =  df_us_age['COUNTY_CODE'].astype(int)
# df_us_age['fips'] = (df_us_age['STATE_FIPS_CODE'].astype(str).astype(str).str.zfill(2) + df_us_age['COUNTY_CODE'].astype(str).astype(str).str.zfill(3)).replace('.0','')
df_us_age['fips'] = df_us_age['STATE_FIPS_CODE'].astype(str).astype(str).str.zfill(2)
df_us_age_clean = df_us_age.sort_values(['fips','CLASS_DESC']).copy().reset_index(drop=True)
fips_age_summary = df_us_age_clean.groupby(['fips', 'CLASS_DESC'], as_index=False)['VALUE_NUMERIC'].sum()
# Compute total operations by FIPS explicitly
fips_totals = fips_age_summary.groupby('fips')['VALUE_NUMERIC'].transform('sum')

# Add percentage explicitly
fips_age_summary['percent_of_ops'] = fips_age_summary['VALUE_NUMERIC'] / fips_totals * 100

# GFR $5M+
# Age 35-50

age_pivot = fips_age_summary.pivot(index='fips', columns='CLASS_DESC', values=['VALUE_NUMERIC', 'percent_of_ops'])

# Calculate explicitly for 35-50
age_pivot['VALUE_NUMERIC', 'AGE 35 TO 50'] = (
    age_pivot['VALUE_NUMERIC', 'AGE 35 TO 44'] +
    0.6 * age_pivot['VALUE_NUMERIC', 'AGE 45 TO 54']
)

age_pivot['percent_of_ops', 'AGE 35 TO 50'] = (
    age_pivot['percent_of_ops', 'AGE 35 TO 44'] +
    0.6 * age_pivot['percent_of_ops', 'AGE 45 TO 54']
)

# Reset and flatten columns for convenience
age_pivot.columns = ['_'.join(col).strip() for col in age_pivot.columns.values]
age_pivot = age_pivot.reset_index()

# View clearly
print(age_pivot[['fips', 'VALUE_NUMERIC_AGE 35 TO 50', 'percent_of_ops_AGE 35 TO 50']])

fips_age_summary = pd.merge(fips_age_summary, age_pivot[['fips', 'VALUE_NUMERIC_AGE 35 TO 50', 'percent_of_ops_AGE 35 TO 50']], on='fips', how='left')

import pandas as pd

# Pivot clearly for calculation ease
age_pivot = fips_age_summary.pivot(index='fips', columns='CLASS_DESC', values=['VALUE_NUMERIC', 'percent_of_ops'])

# Explicit calculation of 40–60 range
age_pivot['VALUE_NUMERIC', 'AGE 40 TO 60'] = (
    0.5 * age_pivot['VALUE_NUMERIC', 'AGE 35 TO 44'] +
    age_pivot['VALUE_NUMERIC', 'AGE 45 TO 54'] +
    0.6 * age_pivot['VALUE_NUMERIC', 'AGE 55 TO 64']
)

age_pivot['percent_of_ops', 'AGE 40 TO 60'] = (
    0.5 * age_pivot['percent_of_ops', 'AGE 35 TO 44'] +
    age_pivot['percent_of_ops', 'AGE 45 TO 54'] +
    0.6 * age_pivot['percent_of_ops', 'AGE 55 TO 64']
)

# Flatten columns explicitly
age_pivot.columns = ['_'.join(col).strip() for col in age_pivot.columns.values]
age_pivot.reset_index(inplace=True)

# Clear view of results
print(age_pivot[['fips', 'VALUE_NUMERIC_AGE 40 TO 60', 'percent_of_ops_AGE 40 TO 60']])

fips_age_summary = pd.merge(fips_age_summary, age_pivot[['fips', 'VALUE_NUMERIC_AGE 40 TO 60', 'percent_of_ops_AGE 40 TO 60']], on='fips', how='left')

import pandas as pd

# Pivot your data for clarity and calculation
age_pivot = fips_age_summary.pivot(index='fips', columns='CLASS_DESC', values=['VALUE_NUMERIC', 'percent_of_ops'])

# Calculate explicitly for 45–65
age_pivot['VALUE_NUMERIC', 'AGE 45 TO 65'] = (
    age_pivot['VALUE_NUMERIC', 'AGE 45 TO 54'] +
    age_pivot['VALUE_NUMERIC', 'AGE 55 TO 64'] +
    0.1 * age_pivot['VALUE_NUMERIC', 'AGE 65 TO 74']
)

age_pivot['percent_of_ops', 'AGE 45 TO 65'] = (
    age_pivot['percent_of_ops', 'AGE 45 TO 54'] +
    age_pivot['percent_of_ops', 'AGE 55 TO 64'] +
    0.1 * age_pivot['percent_of_ops', 'AGE 65 TO 74']
)

# Flatten columns for easier usage
age_pivot.columns = ['_'.join(col).strip() for col in age_pivot.columns.values]
age_pivot.reset_index(inplace=True)

# View results explicitly
print(age_pivot[['fips', 'VALUE_NUMERIC_AGE 45 TO 65', 'percent_of_ops_AGE 45 TO 65']])

fips_age_summary = pd.merge(fips_age_summary, age_pivot[['fips', 'VALUE_NUMERIC_AGE 45 TO 65', 'percent_of_ops_AGE 45 TO 65']], on='fips', how='left')


import pandas as pd

# Pivot data for clarity and easy calculation
age_pivot = fips_age_summary.pivot(index='fips', columns='CLASS_DESC', values=['VALUE_NUMERIC', 'percent_of_ops'])

# Calculate explicitly for 60+
age_pivot['VALUE_NUMERIC', 'AGE 55+'] = (
    age_pivot['VALUE_NUMERIC', 'AGE 55 TO 64'] +
    age_pivot['VALUE_NUMERIC', 'AGE 65 TO 74'] +
    age_pivot['VALUE_NUMERIC', 'AGE GE 75']
)

age_pivot['percent_of_ops', 'AGE 55+'] = (
    age_pivot['percent_of_ops', 'AGE 55 TO 64'] +
    age_pivot['percent_of_ops', 'AGE 65 TO 74'] +
    age_pivot['percent_of_ops', 'AGE GE 75']
)

# Flatten columns for convenience
age_pivot.columns = ['_'.join(col).strip() for col in age_pivot.columns.values]
age_pivot.reset_index(inplace=True)

# View results clearly
print(age_pivot[['fips', 'VALUE_NUMERIC_AGE 55+', 'percent_of_ops_AGE 55+']])

fips_age_summary = pd.merge(fips_age_summary, age_pivot[['fips', 'VALUE_NUMERIC_AGE 55+', 'percent_of_ops_AGE 55+']], on='fips', how='left')


import pandas as pd

# Pivot data for clarity and easy calculation
age_pivot = fips_age_summary.pivot(index='fips', columns='CLASS_DESC', values=['VALUE_NUMERIC', 'percent_of_ops'])

# Calculate explicitly for 60+
age_pivot['VALUE_NUMERIC', 'AGE 60+'] = (
    0.5 * age_pivot['VALUE_NUMERIC', 'AGE 55 TO 64'] +
    age_pivot['VALUE_NUMERIC', 'AGE 65 TO 74'] +
    age_pivot['VALUE_NUMERIC', 'AGE GE 75']
)

# Calculate explicitly for 60+
age_pivot['percent_of_ops', 'AGE 60+'] = (
    0.5 * age_pivot['percent_of_ops', 'AGE 55 TO 64'] +
    age_pivot['percent_of_ops', 'AGE 65 TO 74'] +
    age_pivot['percent_of_ops', 'AGE GE 75']
)

# Flatten columns for convenience
age_pivot.columns = ['_'.join(col).strip() for col in age_pivot.columns.values]
age_pivot.reset_index(inplace=True)

# View results clearly
print(age_pivot[['fips', 'VALUE_NUMERIC_AGE 60+', 'percent_of_ops_AGE 60+']])

fips_age_summary = pd.merge(fips_age_summary, age_pivot[['fips', 'VALUE_NUMERIC_AGE 60+', 'percent_of_ops_AGE 60+']], on='fips', how='left')

fips_age_summary_all = (
    df_us_age_clean
    .groupby('fips', as_index=False)
    .agg({'VALUE_NUMERIC': 'sum'})
    .rename(columns={'VALUE_NUMERIC': 'total_ops_all_ages'})
)
fips_age_summary_all['total_ops_all_ages'].sum()

fips_maturity_model_age_distibutions = fips_age_summary[['fips', 'VALUE_NUMERIC_AGE 35 TO 50', 'percent_of_ops_AGE 35 TO 50', 'VALUE_NUMERIC_AGE 40 TO 60', 'percent_of_ops_AGE 40 TO 60', 'VALUE_NUMERIC_AGE 45 TO 65', 'percent_of_ops_AGE 45 TO 65', 'VALUE_NUMERIC_AGE 55+', 'percent_of_ops_AGE 55+', 'VALUE_NUMERIC_AGE 60+', 'percent_of_ops_AGE 60+']].drop_duplicates().copy().reset_index(drop=True)
fips_maturity_model_age_distibutions.to_csv('fips_maturity_model_age_distibutions.csv', index=False)
fips_maturity_model_age_distibutions['VALUE_NUMERIC_AGE 35 TO 50'].sum()
#



In [ ]:
['SOURCE_DESC',
 'SECTOR_DESC',
 'GROUP_DESC',
 'COMMODITY_DESC',
 'CLASS_DESC',
 'PRODN_PRACTICE_DESC',
 'UTIL_PRACTICE_DESC',
 'STATISTICCAT_DESC',
 'UNIT_DESC',
 'SHORT_DESC',
 'DOMAIN_DESC',
 'DOMAINCAT_DESC',
 'AGG_LEVEL_DESC',
 'STATE_ANSI',
 'STATE_FIPS_CODE',
 'STATE_ALPHA',
 'STATE_NAME',
 'ASD_CODE',
 'ASD_DESC',
 'COUNTY_ANSI',
 'COUNTY_CODE',
 'COUNTY_NAME',
 'REGION_DESC']


In [ ]:
operator_field_to_use = 'COMMODITY TOTALS - OPERATIONS WITH SALES'
['NOT SPECIFIED',
 'FARM SALES: (10,000 TO 19,999 $)',
 'FARM SALES: (1,000 TO 2,499 $)',
 'FARM SALES: (40,000 TO 49,999 $)',
 'FARM SALES: (LESS THAN 1,000 $)',
 'FARM SALES: (10,000 TO 24,999 $)',
 'FARM SALES: (100,000 OR MORE $)',
 'FARM SALES: (100,000 TO 249,999 $)',
 'FARM SALES: (25,000 TO 39,999 $)',
 'FARM SALES: (50,000 TO 99,999 $)',
 'FARM SALES: (LESS THAN 2,500 $)',
 'FARM SALES: (500,000 OR MORE $)',
 'FARM SALES: (2,500 TO 4,999 $)',
 'FARM SALES: (250,000 TO 499,999 $)',
 'FARM SALES: (20,000 TO 24,999 $)',
 'FARM SALES: (25,000 TO 49,999 $)',
 'FARM SALES: (5,000 TO 9,999 $)']
below_250k_categories = [
    'FARM SALES: (LESS THAN 1,000 $)',
    'FARM SALES: (1,000 TO 2,499 $)',
    'FARM SALES: (2,500 TO 4,999 $)',
    'FARM SALES: (5,000 TO 9,999 $)',
    'FARM SALES: (10,000 TO 19,999 $)',
    'FARM SALES: (20,000 TO 24,999 $)',
    'FARM SALES: (10,000 TO 24,999 $)', # careful of potential overlap
    'FARM SALES: (25,000 TO 39,999 $)',
    'FARM SALES: (40,000 TO 49,999 $)',
    'FARM SALES: (25,000 TO 49,999 $)', # careful of potential overlap
    'FARM SALES: (50,000 TO 99,999 $)',
    'FARM SALES: (100,000 TO 249,999 $)',
    'FARM SALES: (LESS THAN 2,500 $)' # overlaps with smaller categories, consider carefully
]
above_250k_categories = [
    'FARM SALES: (250,000 TO 499,999 $)',
    'FARM SALES: (500,000 TO 999,999 $)',
    'FARM SALES: (500,000 OR MORE $)',
    'FARM SALES: (1,000,000 OR MORE $)'
]

rev_desc_to_keep = ['FARM SALES: (LESS THAN 1,000 $)', 'FARM SALES: (1,000 TO 2,499 $)', 'FARM SALES: (2,500 TO 4,999 $)', 'FARM SALES: (5,000 TO 9,999 $)', 'FARM SALES: (10,000 TO 19,999 $)', 'FARM SALES: (20,000 TO 24,999 $)', 'FARM SALES: (25,000 TO 39,999 $)', 'FARM SALES: (40,000 TO 49,999 $)', 'FARM SALES: (50,000 TO 99,999 $)', 'FARM SALES: (100,000 TO 249,999 $)', 'FARM SALES: (250,000 TO 499,999 $)', 'FARM SALES: (500,000 TO 999,999 $)', 'FARM SALES: (1,000,000 TO 2,499,999 $)', 'FARM SALES: (2,500,000 TO 4,999,999 $)', 'FARM SALES: (5,000,000 OR MORE $)']


df_us['SECTOR_DESC'].drop_duplicates()
df_us.loc[df_us['SECTOR_DESC'] == 'ECONOMICS', 'GROUP_DESC'].drop_duplicates()
# fips_sales = df_us.loc[((df_us['SHORT_DESC'] == operator_field_to_use) & ((df_us['DOMAINCAT_DESC'].isin(below_250k_categories) | (df_us['DOMAINCAT_DESC'].isin(above_250k_categories)))) & (df_us['AGG_LEVEL_DESC'] == 'COUNTY'))].drop_duplicates().sort_values(['STATE_FIPS_CODE', 'COUNTY_CODE']).copy().reset_index(drop=True)
fips_sales = df_us.loc[((df_us['SHORT_DESC'] == operator_field_to_use)  & (df_us['AGG_LEVEL_DESC'] == 'STATE'))].drop_duplicates().sort_values(['STATE_FIPS_CODE', 'COUNTY_CODE']).copy().reset_index(drop=True)
fips_sales['VALUE_NUMERIC']  = pd.to_numeric(fips_sales['VALUE'].str.replace(',', ''), errors='coerce')
fips_sales

fips_sales.loc[((fips_sales['SECTOR_DESC'] == 'ECONOMICS') & (fips_sales['SHORT_DESC'] == operator_field_to_use)& (fips_sales['DOMAIN_DESC'] == 'FARM SALES')),['STATE_FIPS_CODE','SECTOR_DESC', 'GROUP_DESC', 'SHORT_DESC' ,'VALUE', 'DOMAIN_DESC', 'DOMAINCAT_DESC']].drop_duplicates().sort_values('STATE_FIPS_CODE')

fips_sales.loc[((fips_sales['SECTOR_DESC'] == 'ECONOMICS') & (fips_sales['SHORT_DESC'] == operator_field_to_use)& (fips_sales['DOMAIN_DESC'] == 'FARM SALES') & (fips_sales['DOMAINCAT_DESC'].isin(rev_desc_to_keep))),['VALUE_NUMERIC']].sum()
fips_sales_clean = fips_sales.loc[((fips_sales['SECTOR_DESC'] == 'ECONOMICS') & (fips_sales['SHORT_DESC'] == operator_field_to_use)& (fips_sales['DOMAIN_DESC'] == 'FARM SALES') & (fips_sales['DOMAINCAT_DESC'].isin(rev_desc_to_keep)))].copy()
fips_sales_clean['VALUE_NUMERIC'].sum()
# 3,813,560


# fips_sales.loc[((fips_sales['SECTOR_DESC'] == 'DEMOGRAPHICS') & fips_sales['SECTOR_DESC'] == 'DEMOGRAPHICS')),['VALUE_NUMERIC']].astype(int).sum()
# 11,967,386
fips_sales = fips_sales_clean.copy().reset_index(drop=True)
fips_sales
# fips_sales['COUNTY_CODE'] =  fips_sales['COUNTY_CODE'].astype(int)
fips_sales['fips'] = fips_sales['STATE_FIPS_CODE'].astype(str).astype(str).str.zfill(2)


# fips_sales['fips'] = (fips_sales['STATE_FIPS_CODE'].astype(str).astype(str).str.zfill(2) + fips_sales['COUNTY_CODE'].astype(str).astype(str).str.zfill(3)).replace('.0','')

# fips_sales['DOMAINCAT_DESC'].drop_duplicates()
# fips_sales.loc[fips_sales['DOMAINCAT_DESC'].str.contains('FARM SALES'), 'SHORT_DESC'].drop_duplicates()
# fips_sales.loc[fips_sales['SHORT_DESC'].str.contains('COMMODITY TOTALS - OPERATIONS WITH SALES'), 'DOMAINCAT_DESC'].drop_duplicates().to_list()

fips_sales_pivot = fips_sales.pivot_table(index='fips', columns='DOMAINCAT_DESC', values='VALUE_NUMERIC', aggfunc='sum').reset_index()
fips_sales_pivot


fips_sales_pivot['FARM_SALES_5M_PLUS'] = fips_sales_pivot['FARM SALES: (5,000,000 OR MORE $)']
fips_sales_pivot['FARM_SALES_2_TO_5_MILLION'] = (fips_sales_pivot['FARM SALES: (2,500,000 TO 4,999,999 $)'] + fips_sales_pivot['FARM SALES: (1,000,000 TO 2,499,999 $)'] * (1/3)).round(0).astype(int)
fips_sales_pivot['FARM_SALES_1_TO_2_MILLION'] = (
    # Two-thirds explicitly represent 1M–2M
    fips_sales_pivot['FARM SALES: (1,000,000 TO 2,499,999 $)'] * (2/3)
).round(0).astype(int)
fips_sales_pivot['FARM_SALES_500K_TO_1MILLION'] = fips_sales_pivot['FARM SALES: (500,000 TO 999,999 $)'].round(0).astype(int)

fips_sales_pivot['FARM_SALES_UNDER_500K'] = fips_sales_pivot['FARM SALES: (1,000 TO 2,499 $)'] + fips_sales_pivot['FARM SALES: (2,500 TO 4,999 $)'] + fips_sales_pivot['FARM SALES: (5,000 TO 9,999 $)'] + fips_sales_pivot['FARM SALES: (10,000 TO 19,999 $)'] + fips_sales_pivot['FARM SALES: (20,000 TO 24,999 $)'] + fips_sales_pivot['FARM SALES: (25,000 TO 39,999 $)'] + fips_sales_pivot['FARM SALES: (40,000 TO 49,999 $)'] + fips_sales_pivot['FARM SALES: (50,000 TO 99,999 $)'] + fips_sales_pivot['FARM SALES: (100,000 TO 249,999 $)'] + fips_sales_pivot['FARM SALES: (250,000 TO 499,999 $)']
fips_sales_pivot['FARM_SALES_UNDER_250K'] = fips_sales_pivot['FARM SALES: (1,000 TO 2,499 $)'] + fips_sales_pivot['FARM SALES: (2,500 TO 4,999 $)'] + fips_sales_pivot['FARM SALES: (5,000 TO 9,999 $)'] + fips_sales_pivot['FARM SALES: (10,000 TO 19,999 $)'] + fips_sales_pivot['FARM SALES: (20,000 TO 24,999 $)'] + fips_sales_pivot['FARM SALES: (25,000 TO 39,999 $)'] + fips_sales_pivot['FARM SALES: (40,000 TO 49,999 $)'] + fips_sales_pivot['FARM SALES: (50,000 TO 99,999 $)'] + fips_sales_pivot['FARM SALES: (100,000 TO 249,999 $)']
fips_sales_pivot['FARM_SALES_OVER_250K'] = fips_sales_pivot['FARM SALES: (250,000 TO 499,999 $)'] + fips_sales_pivot['FARM SALES: (500,000 TO 999,999 $)'] + fips_sales_pivot['FARM SALES: (1,000,000 TO 2,499,999 $)'] + fips_sales_pivot['FARM SALES: (2,500,000 TO 4,999,999 $)'] + fips_sales_pivot['FARM SALES: (5,000,000 OR MORE $)']
fips_sales_pivot
fips_sales_state_summary = fips_sales_pivot[['fips', 'FARM_SALES_UNDER_250K', 'FARM_SALES_OVER_250K', 'FARM_SALES_UNDER_500K', 'FARM_SALES_500K_TO_1MILLION', 'FARM_SALES_1_TO_2_MILLION', 'FARM_SALES_2_TO_5_MILLION', 'FARM_SALES_5M_PLUS']]


fips_sales_pivot['FARM_SALES_OVER_250K'].sum()

fips_sales_state_summary.to_csv('fips_sales_breaks.csv', index=False)
# 1,900,487


In [ ]:
(fips_sales_pivot['FARM_SALES_5M_PLUS']).sum()

In [ ]:
fips_sales_with_age = pd.merge(fips_sales_state_summary, fips_maturity_model_age_distibutions[['fips', 'percent_of_ops_AGE 35 TO 50', 'percent_of_ops_AGE 40 TO 60', 'percent_of_ops_AGE 45 TO 65', 'percent_of_ops_AGE 55+','percent_of_ops_AGE 60+']], on='fips', how='left').copy().reset_index(drop=True)
fips_sales_with_age.to_csv('fips_sales_with_age.csv', index=False)

fips_sales_with_age['MM_INNOVATORS'] = (fips_sales_with_age['FARM_SALES_5M_PLUS'] * (fips_sales_with_age['percent_of_ops_AGE 35 TO 50']/100)).round(0).astype(int)
fips_sales_with_age['MM_EARLY_ADOPTERS'] = (fips_sales_with_age['FARM_SALES_2_TO_5_MILLION'] * (fips_sales_with_age['percent_of_ops_AGE 40 TO 60']/100)).round(0).astype(int)
fips_sales_with_age['MM_EARLY_MAJORITY'] = (fips_sales_with_age['FARM_SALES_1_TO_2_MILLION'] * (fips_sales_with_age['percent_of_ops_AGE 45 TO 65']/100)).round(0).astype(int)
fips_sales_with_age['MM_LATE_MAJORITY'] = (fips_sales_with_age['FARM_SALES_500K_TO_1MILLION'] * (fips_sales_with_age['percent_of_ops_AGE 55+']/100)).round(0).astype(int)

fips_sales_with_age['MM_LAGGARDS'] = (fips_sales_with_age['FARM_SALES_UNDER_500K'] * (fips_sales_with_age['percent_of_ops_AGE 60+']/100)).round(0).astype(int)
fips_sales_with_age['MM_LAGGARDS'].sum()

fips_sales_with_age.to_csv('fips_sales_with_age_maturity_model.csv', index=False)
#
# 19,713,906
# 617,209


Maturity model for canada

In [ ]:
df_canada.head()

df_prov_sales = df_canada.loc[~df_canada['GEO'].str.contains('Canada') & ~df_canada['GEO'].str.contains(',')]
df_prov_sales['VALUE_NUMERIC']  = pd.to_numeric(df_prov_sales['VALUE'].fillna(0.0).round(0).astype(int), errors='coerce')
df_prov_sales['geo_code'] = df_prov_sales['DGUID'].str[-2:]
df_prov_sales_pivot = df_prov_sales.pivot_table(index='geo_code', columns='Total farm revenues distribution', values='VALUE_NUMERIC', aggfunc='sum').reset_index()
df_prov_sales_pivot


df_prov_sales_pivot['FARM_SALES_5M_PLUS'] = (df_prov_sales_pivot['$2,000,000 and over'] * .24).round(0).astype(int)
df_prov_sales_pivot['FARM_SALES_2_TO_5_MILLION'] = (df_prov_sales_pivot['$2,000,000 and over'] * .76).round(0).astype(int)
df_prov_sales_pivot['FARM_SALES_1_TO_2_MILLION'] = df_prov_sales_pivot['$1,000,000 to $1,999,999']
df_prov_sales_pivot['FARM_SALES_500K_TO_1MILLION'] = df_prov_sales_pivot['$500,000 to $999,999'].round(0).astype(int)

df_prov_sales_pivot['FARM_SALES_UNDER_500K'] = df_prov_sales_pivot['$0'] + df_prov_sales_pivot['$1 to $9,999'] + df_prov_sales_pivot['$10,000 to $24,999'] + df_prov_sales_pivot['$25,000 to $49,999'] + df_prov_sales_pivot['$50,000 to $99,999'] + df_prov_sales_pivot['$100,000 to $249,999'] + df_prov_sales_pivot['$250,000 to $499,999']
df_prov_sales_pivot['FARM_SALES_UNDER_250K'] = df_prov_sales_pivot['$0'] + df_prov_sales_pivot['$1 to $9,999'] + df_prov_sales_pivot['$10,000 to $24,999'] + df_prov_sales_pivot['$25,000 to $49,999'] + df_prov_sales_pivot['$50,000 to $99,999'] + df_prov_sales_pivot['$100,000 to $249,999']

df_prov_sales_pivot['FARM_SALES_OVER_250K'] = df_prov_sales_pivot['$250,000 to $499,999'] + df_prov_sales_pivot['$500,000 to $999,999'] + df_prov_sales_pivot['$1,000,000 to $1,999,999'] + df_prov_sales_pivot['$2,000,000 and over']
df_prov_sales_pivot
df_prov_sales_summary = df_prov_sales_pivot[['geo_code', 'FARM_SALES_UNDER_250K', 'FARM_SALES_OVER_250K', 'FARM_SALES_UNDER_500K', 'FARM_SALES_500K_TO_1MILLION', 'FARM_SALES_1_TO_2_MILLION', 'FARM_SALES_2_TO_5_MILLION', 'FARM_SALES_5M_PLUS']]

df_prov_sales_pivot.sum()

df_prov_sales_pivot['FARM_SALES_OVER_250K'].sum()



In [ ]:
(df_prov_sales_pivot['FARM_SALES_5M_PLUS'] ).sum()

In [ ]:
ca_age_bands = ['Age - 35 to 54 years',
 'Age - 55 years and over',
 'Age - under 35 years',
                'Total number of farms'
]
['VALUE_NUMERIC_AGE 35 TO 50', 'percent_of_ops_AGE 35 TO 50', 'VALUE_NUMERIC_AGE 40 TO 60', 'percent_of_ops_AGE 40 TO 60', 'VALUE_NUMERIC_AGE 45 TO 65', 'percent_of_ops_AGE 45 TO 65', 'VALUE_NUMERIC_AGE 55+', 'percent_of_ops_AGE 55+', 'VALUE_NUMERIC_AGE 60+', 'percent_of_ops_AGE 60+']
df_cana_farmer_age = sc.table_to_df('32-10-0381-01')

df_cana_farmer_age = df_cana_farmer_age.loc[~df_cana_farmer_age['GEO'].str.contains('Canada') & ~df_cana_farmer_age['GEO'].str.contains(',') & df_cana_farmer_age['Farms according to the number of operators reported'].str.contains('All') & df_cana_farmer_age['Characteristics'].isin(ca_age_bands)]
df_cana_farmer_age['VALUE_NUMERIC']  = pd.to_numeric(df_cana_farmer_age['VALUE'].fillna(0.0).round(0).astype(int), errors='coerce')
df_cana_farmer_age['geo_code'] = df_cana_farmer_age['DGUID'].str[-2:]
# df_cana_farmer_age['VALUE_NUMERIC']  = pd.to_numeric(df_cana_farmer_age['VALUE'].fillna(0.0).round(0).astype(int), errors='coerce')
df_cana_farmer_age['Characteristics'].drop_duplicates().sort_values().to_list()
df_cana_farmer_age_pivot = df_cana_farmer_age.pivot_table(index='geo_code', columns='Characteristics', values='VALUE_NUMERIC', aggfunc='sum').reset_index()
# df_cana_farmer_age_pivot

# 35–50
factor_35_50 = 0.69
df_cana_farmer_age_pivot['VALUE 35-50'] = (df_cana_farmer_age_pivot['Age - 35 to 54 years'] * factor_35_50).fillna(0.0).round().astype(int)
df_cana_farmer_age_pivot['PCT 35-50'] = (df_cana_farmer_age_pivot['VALUE 35-50'] / df_cana_farmer_age_pivot['Total number of farms'] * 100).fillna(0.0).astype(float)

# 40–60
factor_40_60 = 0.646
df_cana_farmer_age_pivot['VALUE 40-60'] = (df_cana_farmer_age_pivot['Age - 35 to 54 years'] * factor_40_60 * (39.5/100) + df_cana_farmer_age_pivot['Age - 55 years and over'] * factor_40_60 * (14.1/100)).fillna(0.0).round().astype(int)
df_cana_farmer_age_pivot['PCT 40-60'] = (df_cana_farmer_age_pivot['VALUE 40-60'] / df_cana_farmer_age_pivot['Total number of farms'] * 100).fillna(0.0).astype(float)

# 45–65
factor_45_65 = 0.487
df_cana_farmer_age_pivot['VALUE 45-65'] = (df_cana_farmer_age_pivot['Age - 35 to 54 years'] * factor_45_65 * (39.5/100) + df_cana_farmer_age_pivot['Age - 55 years and over'] * factor_45_65 * ((14.1+5.6)/100)).fillna(0.0).round().astype(int)
df_cana_farmer_age_pivot['PCT 45-65'] = (df_cana_farmer_age_pivot['VALUE 45-65'] / df_cana_farmer_age_pivot['Total number of farms'] * 100).fillna(0.0).astype(float)

# 55+
# Already in dataset
df_cana_farmer_age_pivot['VALUE 55+'] = (df_cana_farmer_age_pivot['Age - 55 years and over'] ).fillna(0.0).round().astype(int)
df_cana_farmer_age_pivot['PCT 55+'] = (df_cana_farmer_age_pivot['VALUE 55+'] / df_cana_farmer_age_pivot['Total number of farms'] * 100).fillna(0.0).astype(float)


# 60+
factor_60_plus = 0.75
df_cana_farmer_age_pivot['VALUE 60+'] = (df_cana_farmer_age_pivot['Age - 55 years and over'] * factor_60_plus).fillna(0.0).round().astype(int)
df_cana_farmer_age_pivot['PCT 60+'] = (df_cana_farmer_age_pivot['VALUE 60+'] / df_cana_farmer_age_pivot['Total number of farms'] * 100).fillna(0.0).astype(float)

df_cana_farmer_age_pivot.to_csv('ca_farmer_age_pivot')

In [ ]:
ca_maturity_model = pd.merge(df_prov_sales_summary, df_cana_farmer_age_pivot, on='geo_code', how='left', suffixes=('', '_age')).copy().reset_index(drop=True)
ca_maturity_model

ca_maturity_model['MM_INNOVATORS'] = (ca_maturity_model['FARM_SALES_5M_PLUS'] * (ca_maturity_model['PCT 35-50']/100)).round(0).astype(int)
ca_maturity_model['MM_EARLY_ADOPTERS'] = (ca_maturity_model['FARM_SALES_2_TO_5_MILLION'] * (ca_maturity_model['PCT 40-60']/100)).round(0).astype(int)
ca_maturity_model['MM_EARLY_MAJORITY'] = (ca_maturity_model['FARM_SALES_1_TO_2_MILLION'] * (ca_maturity_model['PCT 45-65']/100)).round(0).astype(int)
ca_maturity_model['MM_LATE_MAJORITY'] = (ca_maturity_model['FARM_SALES_500K_TO_1MILLION'] * (ca_maturity_model['PCT 55+']/100)).round(0).astype(int)

ca_maturity_model['MM_LAGGARDS'] = (ca_maturity_model['FARM_SALES_UNDER_500K'] * (ca_maturity_model['PCT 60+']/100)).round(0).astype(int)
ca_maturity_model['MM_LAGGARDS'].sum()

ca_maturity_model.to_csv('ca_sales_with_age_maturity_model.csv', index=False)


In [ ]:
# Your service account data loaded into a dictionary
# ca_tractor_census = 'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=3210022901&latestN=5&startDate=&endDate=&csvLocale=en&selectedMembers=%5B%5B1%5D%2C%5B%5D%5D&checkedLevels=1D1'
['SOURCE_DESC',
 'SECTOR_DESC',
 'GROUP_DESC',
 'COMMODITY_DESC',
 'CLASS_DESC',
 'PRODN_PRACTICE_DESC',
 'UTIL_PRACTICE_DESC',
 'STATISTICCAT_DESC',
 'UNIT_DESC',
 'SHORT_DESC',
 'DOMAIN_DESC',
 'DOMAINCAT_DESC',
 'AGG_LEVEL_DESC',
 'STATE_ANSI',
 'STATE_FIPS_CODE',
 'STATE_ALPHA',
 'STATE_NAME',
 'ASD_CODE',
 'ASD_DESC',
 'COUNTY_ANSI',
 'COUNTY_CODE',
 'COUNTY_NAME',
 'REGION_DESC']



In [ ]:
over_10_acreage_categories = [
    'AREA OPERATED: (70.0 TO 99.9 ACRES)',
       'AREA OPERATED: (100 TO 139 ACRES)',
       'AREA OPERATED: (140 TO 179 ACRES)',
       'AREA OPERATED: (10.0 TO 49.9 ACRES)',
       'AREA OPERATED: (50.0 TO 69.9 ACRES)',
       'AREA OPERATED: (2,000 OR MORE ACRES)',
       'AREA OPERATED: (180 TO 219 ACRES)',
       'AREA OPERATED: (500 TO 999 ACRES)',
       'AREA OPERATED: (220 TO 259 ACRES)',
       'AREA OPERATED: (260 TO 499 ACRES)',
       'AREA OPERATED: (1,000 TO 1,999 ACRES)'
]

In [ ]:

df_us['SECTOR_DESC'].drop_duplicates()
equip_settings = ['TRACTORS', 'SELF PROPELLED', 'MACHINERY, OTHER']
df_us.loc[df_us['SECTOR_DESC'] == 'DEMOGRAPHICS', 'GROUP_DESC'].drop_duplicates().sort_values()
df_us.loc[((df_us['SECTOR_DESC'] == 'DEMOGRAPHICS') & (df_us['GROUP_DESC'] == 'FARMS & LAND & ASSETS')), ['CLASS_DESC']].drop_duplicates().sort_values('CLASS_DESC')

df_us.loc[df_us['CLASS_DESC'].str.contains('A'), 'SHORT_DESC'].drop_duplicates()
#
'YEARS'


In [ ]:

# county_df = df.dropna(subset=['COUNTY_CODE', 'STATE_FIPS_CODE']).copy().reset_index(drop=True)
# county_df['COUNTY_FIPS'] = county_df['STATE_FIPS_CODE'].astype(str).str.zfill(2) + \
#                            county_df['COUNTY_CODE'].astype(int).astype(str).str.zfill(3)
operator_field_to_use = 'COMMODITY TOTALS - OPERATIONS WITH SALES'

In [ ]:
# @title us over and under 250k with diy
# summarize usda farm operations data
state_df = None
county_df = None
national_df = None
state_summary = None
county_summary = None
national_summary = None

below_250k_categories = [
    'FARM SALES: (LESS THAN 1,000 $)',
    'FARM SALES: (1,000 TO 2,499 $)',
    'FARM SALES: (2,500 TO 4,999 $)',
    'FARM SALES: (5,000 TO 9,999 $)',
    'FARM SALES: (10,000 TO 24,999 $)',
    'FARM SALES: (25,000 TO 49,999 $)',
    'FARM SALES: (50,000 TO 99,999 $)',
    'FARM SALES: (100,000 TO 249,999 $)'
]

above_250k_categories = [
    'FARM SALES: (250,000 TO 499,999 $)',
    'FARM SALES: (500,000 TO 999,999 $)',
    'FARM SALES: (1,000,000 OR MORE $)'
]


state_df = df_us[
    (df_us['SHORT_DESC'] == operator_field_to_use) &
    (df_us['AGG_LEVEL_DESC'] == 'STATE') &
    (df_us['YEAR'] == 2022)
].copy()


state_df['VALUE_NUMERIC'] = pd.to_numeric(state_df['VALUE'].str.replace(',', ''), errors='coerce')
state_df.columns.to_list()
# Aggregate clearly by state
state_df['STATEFP'] = state_df['STATE_FIPS_CODE'].astype(int).astype(str).str.zfill(2)
state_summary = state_df.groupby(['STATEFP','STATE_NAME']).apply(lambda x: pd.Series({
    'Ops_below_250k': x[x['DOMAINCAT_DESC'].isin(below_250k_categories)]['VALUE_NUMERIC'].sum(),
    'Ops_250k_or_more': x[x['DOMAINCAT_DESC'].isin(above_250k_categories)]['VALUE_NUMERIC'].sum()
}), include_groups=False).reset_index()

# Add total clearly
state_summary['Total_Ops'] = state_summary['Ops_below_250k'] + state_summary['Ops_250k_or_more']
state_summary['total_population'] = state_summary['Total_Ops']

state_summary = state_summary.rename(columns={'STATEFP': 'geo_code', 'STATE_NAME': 'geo_name'})

us_over250k_farm_diy_spi_index = estimate_search_population_indexes(us_over250_paid_trends, state_summary, baseline_audince_factor=0.75, interest_multiplier=5)
us_over250k_farm_diy_spi_index['Ops_over_250k_diy_factor'] = us_over250k_farm_diy_spi_index['estimated_audience_factor']


us_under250k_farm_diy_spi_index = estimate_search_population_indexes(us_under_250_paid_trends, state_summary, baseline_audince_factor=0.6, interest_multiplier=5)
us_under250k_farm_diy_spi_index['Ops_below_250k_diy_factor'] = us_under250k_farm_diy_spi_index['estimated_audience_factor']


us_farm_ops_by_state_w_diy = pd.merge(state_summary, us_over250k_farm_diy_spi_index[['geo_code', 'Ops_over_250k_diy_factor']], on='geo_code', how='left', suffixes=('', '_spi'))
us_farm_ops_by_state_w_diy = pd.merge(us_farm_ops_by_state_w_diy, us_under250k_farm_diy_spi_index[['geo_code', 'Ops_below_250k_diy_factor']], on='geo_code', how='left', suffixes=('', '_uspi'))
# state_farm_ops_by_state_w_diy['Ops_below_250k_diy'] = (state_farm_ops_by_state_w_diy['Ops_below_250k'] * state_farm_ops_by_state_w_diy['SPI']).round(0).astype(int)
us_farm_ops_by_state_w_diy['Ops_below_250k_diy'] = (us_farm_ops_by_state_w_diy['Ops_below_250k'] * us_farm_ops_by_state_w_diy['Ops_below_250k_diy_factor']).round(0).astype(int)
us_farm_ops_by_state_w_diy['Ops_250k_or_more_diy'] = (us_farm_ops_by_state_w_diy['Ops_250k_or_more'] * us_farm_ops_by_state_w_diy['Ops_over_250k_diy_factor']).round(0).astype(int)



us_farm_ops_by_state_w_diy.to_csv('us_farm_ops_by_state_w_diy.csv', index=False)


us_over250k_farm_diy_spi_index.to_csv('us_over250k_diy_spi_index.csv', index=False)
us_under250k_farm_diy_spi_index.to_csv('us_under250k_diy_spi_index.csv', index=False)




county_df = df_us[
    (df_us['SHORT_DESC'] == operator_field_to_use) &
    (df_us['AGG_LEVEL_DESC'] == 'COUNTY') &
    (df_us['YEAR'] == 2022)
].copy()

county_df['VALUE_NUMERIC'] = pd.to_numeric(county_df['VALUE'].str.replace(',', ''), errors='coerce')

# Construct County FIPS explicitly
try:
    county_df['FIPS'] = county_df['STATE_FIPS_CODE'].astype(str).str.zfill(2) + \
                        county_df['COUNTY_CODE'].astype(int).astype(str).str.zfill(3)
except ValueError:
    print(ValueError)

county_summary = county_df.groupby(['STATE_NAME', 'COUNTY_NAME', 'FIPS']).apply(lambda x: pd.Series({
    'Ops_below_250k': x[x['DOMAINCAT_DESC'].isin(below_250k_categories)]['VALUE_NUMERIC'].sum(),
    'Ops_250k_or_more': x[x['DOMAINCAT_DESC'].isin(above_250k_categories)]['VALUE_NUMERIC'].sum()
}), include_groups=False).reset_index()

# Calculate total explicitly
county_summary['Total_Ops'] = county_summary['Ops_below_250k'] + county_summary['Ops_250k_or_more']

print(county_summary.head(10))
national_summary = df_us[
    (df_us['SHORT_DESC'] == operator_field_to_use) &
    (df_us['AGG_LEVEL_DESC'] == 'NATIONAL') &
    (df_us['YEAR'] == 2022)
].copy()

national_summary['VALUE_NUMERIC'] = pd.to_numeric(national_summary['VALUE'].str.replace(',', ''), errors='coerce')

national_below_250k = national_summary[
    national_summary['DOMAINCAT_DESC'].isin(below_250k_categories)
]['VALUE_NUMERIC'].sum()

national_above_250k = national_summary[
    national_summary['DOMAINCAT_DESC'].isin(above_250k_categories)
]['VALUE_NUMERIC'].sum()

total_national_ops = national_below_250k + national_above_250k

print(f"National Operations < $250k: {int(national_below_250k):,}")
print(f"National Operations ≥ $250k: {int(national_above_250k):,}")
print(f"National Total Operations: {int(total_national_ops):,}")

canada large - small

In [ ]:

provincial_summary_canada
# Add total clearly

provincial_summary_canada['total_population'] = provincial_summary_canada['Total_Ops']

# state_summary = state_summary.rename(columns={'STATEFP': 'geo_code', 'STATE_NAME': 'geo_name'})
ca_over250k_farm_diy_spi_index = estimate_search_population_indexes(ca_over250_paid_trends, provincial_summary_canada, baseline_audince_factor=0.75, interest_multiplier=5)
ca_over250k_farm_diy_spi_index['Ops_over_250k_diy_factor'] = ca_over250k_farm_diy_spi_index['estimated_audience_factor']


ca_under250k_farm_diy_spi_index = estimate_search_population_indexes(ca_under250_paid_trends, provincial_summary_canada, baseline_audince_factor=0.6, interest_multiplier=5)
ca_under250k_farm_diy_spi_index['Ops_below_250k_diy_factor'] = ca_under250k_farm_diy_spi_index['estimated_audience_factor']
missing_ca_geos = set(provincial_summary_canada['geo_code'])- set(ca_over250k_farm_diy_spi_index['geo_code'])
default_over250k_diy_factor = 0.75
missing_ca_data = pd.DataFrame(columns=ca_over250k_farm_diy_spi_index.columns)
for geo in missing_ca_geos:
    rec_df = pd.DataFrame(columns=ca_over250k_farm_diy_spi_index.columns)
    rec = {
        'geo_code': geo,
        'Ops_over_250k_diy_factor': default_over250k_diy_factor,
    }
    missing_ca_data = pd.concat([missing_ca_data, pd.DataFrame([rec])], ignore_index=True)

ca_over250k_farm_diy_spi_index = pd.concat([ca_over250k_farm_diy_spi_index, missing_ca_data], ignore_index=True)

default_over250k_diy_factor = 0.6
missing_ca_data = pd.DataFrame(columns=ca_under250k_farm_diy_spi_index.columns)
for geo in missing_ca_geos:
    rec_df = pd.DataFrame(columns=ca_under250k_farm_diy_spi_index.columns)
    rec = {
        'geo_code': geo,
        'Ops_below_250k_diy_factor': default_over250k_diy_factor,
    }
    missing_ca_data = pd.concat([missing_ca_data, pd.DataFrame([rec])], ignore_index=True)

ca_under250k_farm_diy_spi_index = pd.concat([ca_under250k_farm_diy_spi_index, missing_ca_data], ignore_index=True)




# provincial_summary_canada['geo_code'] = provincial_summary_canada['geo_code'].str.replace('PR', '')
all_ca_geo_names = provincial_summary_canada['geo_name'].to_list()

ca_under250k_farm_diy_spi_index

ca_farm_ops_by_province_w_diy = pd.merge(provincial_summary_canada, ca_over250k_farm_diy_spi_index[['geo_code', 'Ops_over_250k_diy_factor']], on='geo_code', how='left', suffixes=('', '_spi'))
ca_farm_ops_by_province_w_diy = pd.merge(ca_farm_ops_by_province_w_diy, ca_under250k_farm_diy_spi_index[['geo_code', 'Ops_below_250k_diy_factor']], on='geo_code', how='left', suffixes=('', '_uspi'))
# state_farm_ops_by_state_w_diy['Ops_below_250k_diy'] = (state_farm_ops_by_state_w_diy['Ops_below_250k'] * state_farm_ops_by_state_w_diy['SPI']).round(0).astype(int)
ca_farm_ops_by_province_w_diy['Ops_below_250k_diy'] = (ca_farm_ops_by_province_w_diy['Ops_below_250k'] * ca_farm_ops_by_province_w_diy['Ops_below_250k_diy_factor']).round(0).astype(int)
ca_farm_ops_by_province_w_diy['Ops_250k_or_more_diy'] = (ca_farm_ops_by_province_w_diy['Ops_250k_or_more'] * ca_farm_ops_by_province_w_diy['Ops_over_250k_diy_factor']).round(0).astype(int)



ca_farm_ops_by_province_w_diy.to_csv('ca_farm_ops_by_province_w_diy.csv', index=False)


ca_over250k_farm_diy_spi_index.to_csv('ca_over250k_diy_spi_index.csv', index=False)
ca_under250k_farm_diy_spi_index.to_csv('ca_under250k_diy_spi_index.csv', index=False)



In [ ]:




density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)
data_col='adjusted_audience_Ops_250k_or_more'
leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
title_txt = f'Audience Sizing for DIY Interest: US Farm Operations > $250k'



# # us_large_search_trends = get_trends_via_scrapingbee(us_over_250k_keywords, scrapingbee_api_key)
# us_large_search_trends_flat = us_large_search_trends.reset_index()
# us_large_trends_prepped = prep_trends_data(us_large_search_trends_flat, us_all_regions, neutral_factor=0.5)


# us_large_paid_prepped = prep_paid_search_data(us_large_paid, us_all_regions, neutral_factor=0.5)

# counts_col='Ops_250k_or_more'
# us_large_audiences = add_us_stats_and_geos(repositioned_us, us_large_trends_prepped, state_summary, us_large_paid_prepped, counts_col=counts_col)
# us_large_audiences = us_large_audiences.to_crs(epsg=5070)
# vert_max = us_large_audiences['adjusted_audience_Ops_250k_or_more'].max()
# data_col = 'adjusted_audience_Ops_250k_or_more'
# file_stub = 'us_diy_over250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: US Farm Operations >= $250k'
# footer_txt = 'Data Sources: 2022 USDA Ag Census, Google Ads API, Google Search Trends'

# plot_us_map(us_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# us_large_audiences[['NAME', 'STATEFP',counts_col,data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


# # ca_province_list = canada_provinces_gdf['PRENAME'].drop_duplicates().tolist()
# # ca_search_trends['geoName'] = ca_search_trends['geoName'].apply(normalize_text)
# ca_search_trends_flat = ca_search_trends.reset_index()







# ca_large_trends_prepped = prep_trends_data(ca_search_trends_flat, ca_all_regions, neutral_factor=0.5)

# # ca_large_trends_prepped['region'] = ca_large_trends_prepped['region'].apply(normalize_text)
# ca_large_paid_prepped = prep_paid_search_data(ca_large_paid, ca_all_regions, neutral_factor=0.5)

# ca_large_paid_prepped
# counts_col='Ops_250k_or_more'

# def add_us_stats_and_geos(gdf, df_trends, df_stats, paid_df, gdf_geo_col='NAME', trends_geo_col='region', stats_geo_col='STATE_NAME', paid_geo_col='state', counts_col=None):
#     if counts_col is None:
#         return "Error: counts_col must be provided."
#     gdf[gdf_geo_col] = gdf[gdf_geo_col].apply(normalize_text)
#     df_trends[trends_geo_col] = df_trends[trends_geo_col].apply(normalize_text)
#     df_stats[stats_geo_col] = df_stats[stats_geo_col].apply(normalize_text)
#     paid_df[paid_geo_col] = paid_df[paid_geo_col].apply(normalize_text)
#     merged_df = gdf.merge(df_trends, left_on=gdf_geo_col, right_on=trends_geo_col, how='left')
#     merged_df = pd.merge(merged_df, df_stats, left_on=gdf_geo_col, right_on=stats_geo_col, how='left', suffixes=('', '_extra'))
#     for col in merged_df.columns:
#         if col.endswith('_extra'):
#             merged_df.drop(columns=[col], inplace=True)

#     merged_df['Ops_below_250k'] = merged_df['Ops_below_250k'].fillna(0).round(0).astype(int)
#     merged_df['Ops_250k_or_more'] = merged_df['Ops_250k_or_more'].fillna(0).round(0).astype(int)
#     merged_df['Total_Ops'] = merged_df['Total_Ops'].fillna(0).round(0).astype(int)
#     comp_col_label = f"{counts_col}_composite_factor_100"
#     merged_df[comp_col_label] = (merged_df[counts_col] / merged_df[counts_col].sum()) * 100
#     # merged_df['Ops_below_250k_composite_factor_100'] = (merged_df['Ops_below_250k'] / merged_df['Ops_250k_or_more'].sum()) * 100
#     merged_df2 = merged_df.merge(paid_df, left_on=gdf_geo_col, right_on=paid_geo_col, how='left').copy().sort_values(gdf_geo_col).reset_index(drop=True)
#     # merged_df2 = merged_df2.loc[~merged_df2[paid_geo_col].isna()].copy().sort_values(gdf_geo_col).reset_index(drop=True)


#     mean_trends = merged_df2['st_composite_factor_100'].fillna(0).mean()
#     mean_volume = merged_df2['paid_search_composite_factor_100'].fillna(0).mean()

#     # Clearly calculate relative positions
#     merged_df2['trends_relative'] = merged_df2['st_composite_factor_100'] / mean_trends
#     merged_df2['volume_relative'] = merged_df2['paid_search_composite_factor_100'] / mean_volume

#     # Clearly combine both into single adjustment factor
#     merged_df2['combined_relative_factor'] = (merged_df2['trends_relative'] + merged_df2['volume_relative']) / 2

#     # Adjusted audience clearly calculated
#     audience_label = f"adjusted_audience_{counts_col}"
#     merged_df2[audience_label] = (merged_df2[counts_col] * merged_df2['combined_relative_factor']).fillna(0).round(0).astype(int)
#     return merged_df2


# ca_large_audiences = add_us_stats_and_geos(canada_provinces_gdf, ca_large_trends_prepped, provincial_summary_canada, ca_large_paid_prepped, counts_col=counts_col, gdf_geo_col='PRENAME',
#                                            stats_geo_col='geo_name').copy()

# ca_large_audiences.columns
# ca_large_audiences = ca_large_audiences.to_crs(epsg=5070)
# vert_max = ca_large_audiences['adjusted_audience_Ops_250k_or_more'].max()
# data_col = 'adjusted_audience_Ops_250k_or_more'
# file_stub = 'ca_diy_over250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: CA Farm Operations >= $250k'
# footer_txt = 'Data Sources: 2021 CA Ag Census, Google Ads API, Google Search Trends'

# plot_us_map(ca_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# ca_large_audiences[['PRENAME', 'PRUID',counts_col,data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)

# na_large_audiences = pd.concat([us_large_audiences[['adjusted_audience_Ops_250k_or_more','Ops_below_250k','Ops_250k_or_more','Total_Ops', 'geometry']], ca_large_audiences[['adjusted_audience_Ops_250k_or_more', 'Ops_below_250k','Ops_250k_or_more','Total_Ops','geometry']]])
# vert_max = na_large_audiences['adjusted_audience_Ops_250k_or_more'].max()
# ile_stub = 'na_diy_over250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: NA Farm Operations >= $250k'
# footer_txt = 'Data Sources: 2022 USDA Ag Census, 2021 CA Ag Census, Google Ads API, Google Search Trends'
# plot_us_map(na_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

In [ ]:
# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'All US Farm Operations'
# footer_txt = 'Data Source: 2022 USDA Ag Census'
# vert_max = us_large_audiences[data_col].max()
# plot_us_map(us_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'All CA Farm Operations'
# footer_txt = 'Data Source: 2021 CA Ag Census'
# vert_max = ca_large_audiences[data_col].max()
# plot_us_map(ca_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'All North American Farm Operations'
# footer_txt = 'Data Source: 2022 USDA Ag Census & 2021 CA Ag Census'
# vert_max = na_large_audiences[data_col].max()
# plot_us_map(na_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)


# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)
# data_col='Ops_250k_or_more'
# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'US Farm Operations Over $250k'
# footer_txt = 'Data Source: 2022 USDA Ag Census'
# vert_max = us_large_audiences[data_col].max()
# plot_us_map(us_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'CA Farm Operations Over $250k'
# footer_txt = 'Data Source: 2021 CA Ag Census'
# vert_max = ca_large_audiences[data_col].max()
# plot_us_map(ca_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'North American Farm Operations Over $250k'
# footer_txt = 'Data Source: 2022 USDA Ag Census & 2021 CA Ag Census'
# vert_max = na_large_audiences[data_col].max()
# plot_us_map(na_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)



# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)
# data_col='Ops_below_250k'
# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'US Farm Operations Under $250k'
# footer_txt = 'Data Source: 2022 USDA Ag Census'
# vert_max = us_large_audiences[data_col].max()
# plot_us_map(us_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'CA Farm Operations Under $250k'
# footer_txt = 'Data Source: 2021 CA Ag Census'
# vert_max = ca_large_audiences[data_col].max()
# plot_us_map(ca_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)

# # @title Over / Under 250k and total maps and data
# density_colors = ['#E0EDF7', '#5E96AE', '#237D83', '#1A4599', '#F36D32']

# density_cmap = LinearSegmentedColormap.from_list('audience', density_colors)

# leg_kwds_dict={'label': "Total Operations", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'North American Farm Operations Under $250k'
# footer_txt = 'Data Source: 2022 USDA Ag Census & 2021 CA Ag Census'
# vert_max = na_large_audiences[data_col].max()
# plot_us_map(na_large_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)



In [ ]:
# #  start on small operators
# us_under_250k_diy_keywords = [
#     'John Deere mower repair',
#     'John Deere lawn tractor parts',
#     'John Deere mower maintenance',
#     'small tractor repair services',
#     'small tractor troubleshooting',
#     'john deere parts'
# ]
# ca_under_250k_diy_keywords = [
#     'John Deere mower repair',
#     'John Deere lawn tractor parts',
#     'John Deere mower maintenance',
#     'small tractor repair services',
#     'small tractor troubleshooting',
#     'John Deere parts',
#     'John Deere snow blower parts',       # Explicitly relevant due to snow conditions
#     'John Deere snow blower repair',      # Frequent seasonal repairs
#     'garden tractor snow blade',          # Attachments specifically useful in Canada
#     'compact tractor winter maintenance',  # Maintenance specific to colder climates
#     'réparation tondeuse John Deere',
#     'pièces tracteur pelouse John Deere'
# ]
# us_small_search_trends = load_trends_from_search_api(us_under_250k_diy_keywords, search_api_io_key)
# us_small_paid = get_keyword_estimates(client, us_under_250k_diy_keywords, us_states)
# ca_small_search_trends = load_trends_from_search_api(ca_under_250k_diy_keywords, search_api_io_key, geo='CA')
# ca_small_paid = get_keyword_estimates(client, ca_under_250k_diy_keywords, ca_provinces)

In [ ]:


# us_small_search_trends_flat = us_small_search_trends.reset_index()
# us_small_op_trends_prepped = prep_trends_data(us_small_search_trends_flat, us_all_regions, neutral_factor=0.5)
# us_small_op_trends_prepped
# us_small_op_paid_prepped = prep_paid_search_data(us_small_paid, us_all_regions, neutral_factor=0.5)
# us_small_op_paid_prepped
# counts_col='Ops_below_250k'
# us_small_op_audiences = add_us_stats_and_geos(repositioned_us, us_small_op_trends_prepped, state_summary, us_small_op_paid_prepped, counts_col=counts_col)
# sus_mall_op_audiences = us_small_op_audiences.to_crs(epsg=5070)
# us_small_op_audiences.loc[us_small_op_audiences['NAME'] == 'PUERTO RICO']
# vert_max = us_small_op_audiences['adjusted_audience_Ops_below_250k'].max()
# data_col = 'adjusted_audience_Ops_below_250k'
# file_stub = 'us_diy_under250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: US Farm Operations < $250k'
# footer_txt = 'Data Sources: USDA, Google Ads API, Google Trends (Data normalized individually then summed)'

# plot_us_map(us_small_op_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# us_small_op_audiences[['NAME', 'STATEFP',counts_col,data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


# ca_small_search_trends_flat = ca_small_search_trends.reset_index()
# ca_small_op_trends_prepped = prep_trends_data(ca_small_search_trends_flat, ca_all_regions, neutral_factor=0.5)
# ca_small_op_trends_prepped
# ca_small_op_paid_prepped = prep_paid_search_data(ca_small_paid, ca_all_regions, neutral_factor=0.5)
# ca_small_op_paid_prepped
# counts_col='Ops_below_250k'
# ca_small_op_audiences = add_us_stats_and_geos(canada_provinces_gdf, ca_small_op_trends_prepped, provincial_summary_canada, ca_small_op_paid_prepped, counts_col=counts_col, gdf_geo_col='PRENAME',
#                                            stats_geo_col='geo_name')
# ca_small_op_audiences = ca_small_op_audiences.to_crs(epsg=5070)
# vert_max = ca_small_op_audiences['adjusted_audience_Ops_below_250k'].max()
# data_col = 'adjusted_audience_Ops_below_250k'
# file_stub = 'ca_diy_under250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: US Farm Operations < $250k'
# footer_txt = 'Data Sources: 2021 CA Ag Census, Google Ads API, Google Search Trends'

# plot_us_map(ca_small_op_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# ca_small_op_audiences[['PRENAME', 'PRUID',counts_col,data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


# na_small_audiences = pd.concat([us_small_op_audiences[['adjusted_audience_Ops_below_250k', 'geometry']], ca_small_op_audiences[['adjusted_audience_Ops_below_250k', 'geometry']]])
# vert_max = na_small_audiences['adjusted_audience_Ops_below_250k'].max()
# file_stub = 'na_diy_below_250k'
# leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
# title_txt = f'Audience Sizing for DIY Interest: NA Farm Operations  $250k'
# footer_txt = 'Data Sources: 2022 USDA Ag Census, 2021 CA Ag Census, Google Ads API, Google Search Trends'
# plot_us_map(na_small_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)


MOVING on to rural lifestylers

In [ ]:
df_us['DOMAIN_DESC'].drop_duplicates().to_list()

In [ ]:
df_acreage = (df_us[(df_us['SHORT_DESC'].str.contains("AREA OPERATED")) &
        (df_us['DOMAIN_DESC'] == "AREA OPERATED") &
        (df_us['AGG_LEVEL_DESC'] == "STATE") &
        (df_us['YEAR'] == 2022)])
df_acreage['DOMAINCAT_DESC'].unique()

In [ ]:
split_acreage_categories = [
    'AREA OPERATED: (1.0 TO 9.9 ACRES)'
]
over_10_acreage_categories = [
    'AREA OPERATED: (70.0 TO 99.9 ACRES)',
       'AREA OPERATED: (100 TO 139 ACRES)',
       'AREA OPERATED: (140 TO 179 ACRES)',
       'AREA OPERATED: (10.0 TO 49.9 ACRES)',
       'AREA OPERATED: (50.0 TO 69.9 ACRES)',
       'AREA OPERATED: (2,000 OR MORE ACRES)',
       'AREA OPERATED: (180 TO 219 ACRES)',
       'AREA OPERATED: (500 TO 999 ACRES)',
       'AREA OPERATED: (220 TO 259 ACRES)',
       'AREA OPERATED: (260 TO 499 ACRES)',
       'AREA OPERATED: (1,000 TO 1,999 ACRES)'
]

In [ ]:
operator_fields_to_use = ['AREA OPERATED', 'COMMODITY TOTALS - OPERATIONS WITH SALES']
over_5_acre_factor = 0.35
under_250k_factor = 0.9
increaser_factor = 1.3

below_250k_categories = [
    'FARM SALES: (LESS THAN 1,000 $)',
    'FARM SALES: (1,000 TO 2,499 $)',
    'FARM SALES: (2,500 TO 4,999 $)',
    'FARM SALES: (5,000 TO 9,999 $)',
    'FARM SALES: (10,000 TO 24,999 $)',
    'FARM SALES: (25,000 TO 49,999 $)',
    'FARM SALES: (50,000 TO 99,999 $)',
    'FARM SALES: (100,000 TO 249,999 $)'
]

above_250k_categories = [
    'FARM SALES: (250,000 TO 499,999 $)',
    'FARM SALES: (500,000 TO 999,999 $)',
    'FARM SALES: (1,000,000 OR MORE $)'
]

df_us_rl_raw = df_us[
    (df_us['SHORT_DESC'].isin(operator_fields_to_use)) &
    (df_us['YEAR'] == 2022)].copy().reset_index(drop=True)


df_us_rl_raw['VALUE_NUMERIC'] = pd.to_numeric(df_us_rl_raw['VALUE'].str.replace(',', ''), errors='coerce')


df_us_rl_raw.columns.to_list()

state_rl_df = df_us[
    (df_us['SHORT_DESC'].isin(operator_fields_to_use)) &
    (df_us['AGG_LEVEL_DESC'] == 'STATE') &
    (df_us['YEAR'] == 2022)
].copy()


state_rl_df['VALUE_NUMERIC'] = pd.to_numeric(state_rl_df['VALUE'].str.replace(',', ''), errors='coerce')
state_rl_df.columns.to_list()
# Aggregate clearly by state
state_rl_df['STATEFP'] = state_rl_df['STATE_FIPS_CODE'].astype(int).astype(str).str.zfill(2)
state_summary = state_rl_df.groupby(['STATEFP','STATE_NAME']).apply(lambda x: pd.Series({
    'Ops_below_250k': x[x['DOMAINCAT_DESC'].isin(below_250k_categories)]['VALUE_NUMERIC'].sum(),
    'Ops_250k_or_more': x[x['DOMAINCAT_DESC'].isin(above_250k_categories)]['VALUE_NUMERIC'].sum()
}), include_groups=False).reset_index()

state_land_summary = state_rl_df.groupby(['STATEFP','STATE_NAME']).apply(lambda x: pd.Series({
    'Ops_below_10_acres': x[x['DOMAINCAT_DESC'].isin(split_acreage_categories)]['VALUE_NUMERIC'].sum(),
    'Ops_10_acres_or_more': x[x['DOMAINCAT_DESC'].isin(over_10_acreage_categories)]['VALUE_NUMERIC'].sum()
}), include_groups=False).reset_index()
state_land_summary['Ops_5_acres_or_more_raw'] = ((state_land_summary['Ops_below_10_acres'] * over_5_acre_factor) + state_land_summary['Ops_10_acres_or_more']).round(0)
state_summary = state_summary.merge(state_land_summary[['STATEFP','Ops_5_acres_or_more_raw']], on='STATEFP', how='left')


state_summary['Ops_5_acres_or_more_under250k'] = state_summary['Ops_5_acres_or_more_raw'] - state_summary['Ops_250k_or_more']

# np.minimum((state_summary['Ops_5_acres_or_more_raw'] * under_250k_factor), (state_summary['Ops_below_250k']))
# state_summary['revenue_proportion'] = state_summary['Ops_below_250k'] / (
#     state_summary['Ops_below_250k'] + state_summary['Ops_250k_or_more']
# )

# # Vectorized min operation explicitly using np.minimum
# state_summary['Ops_5_acres_or_more_adjusted_2'] = np.minimum(
#     state_summary['Ops_5_acres_or_more'] * state_summary['revenue_proportion'] * increaser_factor,
#     state_summary['Ops_below_250k'],  # explicitly capped
#     state_summary['Ops_5_acres_or_more']
# )
# state_summary['Ops_5_acres_or_more_adjusted_2'] = min((state_summary['Ops_5_acres_or_more'] * ((state_summary['Ops_below_250k'] / (state_summary['Ops_below_250k'] + state_summary['Ops_250k_or_more'])))* increaser_factor), (state_summary['Ops_250k_or_more']))

state_summary



# Add total clearly
state_summary['Total_Ops'] = state_summary['Ops_below_250k'] + state_summary['Ops_250k_or_more']
state_summary
# print(state_summary.head(10))
# county_df = df_us[
#     (df_us['SHORT_DESC'] == operator_field_to_use) &
#     (df_us['AGG_LEVEL_DESC'] == 'COUNTY') &
#     (df_us['YEAR'] == 2022)
# ].copy()

# county_df['VALUE_NUMERIC'] = pd.to_numeric(county_df['VALUE'].str.replace(',', ''), errors='coerce')

# # Construct County FIPS explicitly
# try:
#     county_df['FIPS'] = county_df['STATE_FIPS_CODE'].astype(str).str.zfill(2) + \
#                         county_df['COUNTY_CODE'].astype(int).astype(str).str.zfill(3)
# except ValueError:
#     print(ValueError)

# county_summary = county_df.groupby(['STATE_NAME', 'COUNTY_NAME', 'FIPS']).apply(lambda x: pd.Series({
#     'Ops_below_250k': x[x['DOMAINCAT_DESC'].isin(below_250k_categories)]['VALUE_NUMERIC'].sum(),
#     'Ops_250k_or_more': x[x['DOMAINCAT_DESC'].isin(above_250k_categories)]['VALUE_NUMERIC'].sum()
# }), include_groups=False).reset_index()

# # Calculate total explicitly
# county_summary['Total_Ops'] = county_summary['Ops_below_250k'] + county_summary['Ops_250k_or_more']

# print(county_summary.head(10))
# national_summary = df_us[
#     (df_us['SHORT_DESC'] == operator_field_to_use) &
#     (df_us['AGG_LEVEL_DESC'] == 'NATIONAL') &
#     (df_us['YEAR'] == 2022)
# ].copy()

# national_summary['VALUE_NUMERIC'] = pd.to_numeric(national_summary['VALUE'].str.replace(',', ''), errors='coerce')

# national_below_250k = national_summary[
#     national_summary['DOMAINCAT_DESC'].isin(below_250k_categories)
# ]['VALUE_NUMERIC'].sum()

# national_above_250k = national_summary[
#     national_summary['DOMAINCAT_DESC'].isin(above_250k_categories)
# ]['VALUE_NUMERIC'].sum()

# total_national_ops = national_below_250k + national_above_250k

# print(f"National Operations < $250k: {int(national_below_250k):,}")
# print(f"National Operations ≥ $250k: {int(national_above_250k):,}")
# print(f"National Total Operations: {int(total_national_ops):,}")

break 1

In [ ]:
us_rl_audiences = repositioned_us.merge(state_summary, left_on='STATEFP', right_on='STATEFP', how='left')


data_col = 'Ops_5_acres_or_more_under250k'
file_stub = 'us_rl_over5_acres_under250k'
leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
title_txt = f'Audience Sizing for Rural Life Stylers: US Farm Operations < $250k and > 5 acres'
footer_txt = 'Data Sources: 2022 USDA Ag Census'
vert_max = us_rl_audiences[data_col].max()
plot_us_map(us_rl_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
counts_col='Ops_below_250k'
us_rl_audiences[['STATE_NAME', 'STATEFP',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)
us_rl_audiences_clean = us_rl_audiences.copy()
us_rl_audiences_clean['geo_code'] = us_rl_audiences_clean['STATEFP']
us_rl_audiences_clean['geo_name'] = us_rl_audiences_clean['STATE_NAME']
us_rl_audiences_clean[['geo_code', 'geo_name', 'Ops_below_250k', 'Ops_250k_or_more', 'Ops_5_acres_or_more_raw', 'Ops_5_acres_or_more_under250k', 'Total_Ops']].to_csv('us_rl_audiences_clean.csv', index=False)


ca_acre_data_url = 'https://www150.statcan.gc.ca/n1/tbl/csv/32100232-eng.zip'
local_filename = 'ca_agriculture_acre.csv'
df_acre_canada = load_or_download_csv(local_filename, ca_acre_data_url)
# df_canada = df_canada.rename(columns={'Total farm revenues distribution': 'revenue_distribution'})
# df_canada['Total farm revenues distribution'].unique()

ca_over_5_under_10_factor = 0.16

df_acre_canada['Total farm area distribution'].drop_duplicates().to_list()
ca_split_acreage_categories = ['Under 10.00 acres']
ca_over_10_acreage_categories = ['10.00 to 69.99 acres',
    '70.00 to 129.99 acres',
    '130.00 to 179.99 acres',
    '180.00 to 239.99 acres',
    '240.00 to 399.99 acres',
    '400.00 to 559.99 acres',
    '560.00 to 759.99 acres',
    '760.00 to 1,119.99 acres',
    '1,120.00 to 1,599.99 acres',
    '1,600.00 to 2,239.99 acres',
    '2,240.00 to 2,879.99 acres',
    '2,880.00 to 3,519.99 acres',
    '3,520.00 acres and over']


geo_level='provincial'
provincial_summary_acre_canada = categorize_revenues(df_acre_canada, geo_level=geo_level, geo_col='GEO', country_geo_val='Canada [000000000]', above_250k_cats=ca_split_acreage_categories, below_250k_cats=ca_over_10_acreage_categories, country='CA', pivot_col='Total farm area distribution')
provincial_summary_acre_canada = extract_2_vectorized_stats(df_acre_canada, geo_level=geo_level, geo_col='GEO',
                                                            country_geo_val='Canada [000000000]', pivot1=ca_split_acreage_categories, pivot2=ca_over_10_acreage_categories, pivot1_label='Ops_under_10_acres', pivot2_label='Ops_10_acres_or_more',
                                                            pivot_total_label='Total_Ops', country='CA', pivot_col='Total farm area distribution', values_col='VALUE', geo_code_col='geo_code', geo_name_col='geo_name', fips_col='PRUID', agg_function='sum')

provincial_summary_acre_canada = provincial_summary_acre_canada.merge(provincial_summary_canada[['PRUID','Ops_250k_or_more']], on='PRUID', how='left')
provincial_summary_acre_canada[data_col] = ((provincial_summary_acre_canada['Ops_under_10_acres']*ca_over_5_under_10_factor)+ provincial_summary_acre_canada['Ops_10_acres_or_more'] - provincial_summary_acre_canada['Ops_250k_or_more']).round(0).astype(int)
print('/nn')
# print(provincial_summary_acre_canada.head())
provincial_summary_acre_canada['PRUID'] = provincial_summary_acre_canada['PRUID'].astype(str)
provincial_summary_acre_canada.head()
ca_rl_over_5_under_250k = canada_provinces_gdf.merge(provincial_summary_acre_canada[['PRUID',data_col]], on='PRUID', how='left')

title_txt = f'Audience Sizing for Rural Life Stylers: CA Farm Operations < $250k and > 5 acres'
footer_txt = 'Data Sources: 2021 CA Ag Census'
vert_max = ca_rl_over_5_under_250k[data_col].max()
file_stub = 'ca_rl_over5_acres_under250k'
plot_us_map(ca_rl_over_5_under_250k, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
ca_rl_over_5_under_250k_small = ca_rl_over_5_under_250k.copy()
ca_rl_over_5_under_250k_small['geo_code'] = ca_rl_over_5_under_250k_small['PRUID'].astype(str)
ca_rl_over_5_under_250k_small['geo_name'] = ca_rl_over_5_under_250k_small['PRENAME']

ca_rl_over_5_under_250k[['PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_province.csv', index=False)
ca_rl_over_5_under_250k_small.columns


na_rl_over_5_under_250k = pd.concat([us_rl_audiences[['STATE_NAME', 'STATEFP',data_col, 'geometry']], ca_rl_over_5_under_250k[['PRENAME', 'PRUID',data_col, 'geometry']]])

title_txt = f'Audience Sizing for Rural Life Stylers: North American Farm Operations < $250k and > 5 acres'
footer_txt = 'Data Sources: 2022 USDA Ag Census, 2021 CA Ag Census'
vert_max = na_rl_over_5_under_250k[data_col].max()
file_stub = 'na_rl_over5_acres_under250k'
plot_us_map(na_rl_over_5_under_250k, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
na_rl_over_5_under_250k[['STATE_NAME', 'STATEFP', 'PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)
na_rl_over_5_under_250k


# PRUID


Recreational Off roaders

In [ ]:
def get_user_interests(client, customer_id, type='AFFINITY'):
    googleads_service = client.get_service("GoogleAdsService")

    query = f"""
        SELECT
          user_interest.resource_name,
          user_interest.user_interest_id,
          user_interest.name
        FROM user_interest
        WHERE user_interest.taxonomy_type = '{type}'
    """

    response = googleads_service.search(customer_id=customer_id, query=query)

    affinity_interests = []
    for row in response:
        affinity_interests.append({
            "resource_name": row.user_interest.resource_name,
            "id": row.user_interest.user_interest_id,
            "name": row.user_interest.name
        })

    return affinity_interests

gads_affinity_interests = get_user_interests(client, gads_account)
gads_affinity_interests
gads_in_market_interests = get_user_interests(client, gads_account, type='IN_MARKET')


In [ ]:
from fuzzywuzzy import process

def fuzzy_match_interests(interests_list, keywords, threshold=75):
    matched = []
    interest_names = [interest['name'] for interest in interests_list]

    for keyword in keywords:
        matches = process.extract(keyword, interest_names, limit=10)
        for name, score in matches:
            if score >= threshold:
                # find original interest dictionary
                interest = next(i for i in interests_list if i['name'] == name)
                if interest not in matched:
                    matched.append(interest)
    return matched

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load sentence-transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example categories
interest_names = [interest['name'] for interest in gads_in_market_interests]
interest_ids = [interest['id'] for interest in gads_in_market_interests]

# Embed your interests
interest_embeddings = model.encode(interest_names, convert_to_tensor=False, show_progress_bar=True)

# Setup FAISS CPU index
dimension = interest_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add vectors to index
index.add(np.array(interest_embeddings))

# Your keywords for off-road / overlanding
keywords = ["air conditioner repair", "air conditioner replacement", "ac checkup"]

# Embed your keywords
keyword_embeddings = model.encode(keywords, convert_to_tensor=False)

# Perform semantic search on CPU
k = 5  # top 5 matches per keyword
D, I = index.search(np.array(keyword_embeddings), k)

# Display matches
for idx, keyword in enumerate(keywords):
    print(f"\nTop matches for '{keyword}':")
    for dist, interest_idx in zip(D[idx], I[idx]):
        print(f"  - {interest_names[interest_idx]} (ID: {interest_ids[interest_idx]}, Distance: {dist:.2f})")



general semantic search * census

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

class SemanticSearcher:
    def __init__(self, model_name='all-MiniLM-L6-v2', use_gpu=False):
        device = 'cuda' if use_gpu and torch.cuda.is_available() else 'cpu'
        self.model = SentenceTransformer(model_name, device=device)
        self.index = None
        self.data_items = []

    def build_index(self, data_items):
        self.data_items = data_items
        embeddings = self.model.encode(data_items, show_progress_bar=True)
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(np.array(embeddings))

    def query(self, query_items, top_k=5):
        query_embeddings = self.model.encode(query_items)
        distances, indices = self.index.search(np.array(query_embeddings), top_k)
        matches = {}
        for idx, query in enumerate(query_items):
            matches[query] = [
                {'item': self.data_items[i], 'distance': float(distances[idx][j])}
                for j, i in enumerate(indices[idx])
            ]
        return matches

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

class ACSSemanticSearcher:
    def __init__(self, model_name='all-MiniLM-L6-v2', use_gpu=False):
        device = 'cuda' if use_gpu and torch.cuda.is_available() else 'cpu'
        self.model = SentenceTransformer(model_name, device=device)
        self.index = None
        self.data = []

    def build_index(self, data):
        self.data = data
        descriptions = [item['description'] for item in data]
        embeddings = self.model.encode(descriptions, show_progress_bar=True)
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(np.array(embeddings))

    def query(self, query_texts, top_k=5):
        query_embeddings = self.model.encode(query_texts)
        distances, indices = self.index.search(np.array(query_embeddings), top_k)
        matches = {}
        for idx, query in enumerate(query_texts):
            matches[query] = [
                {
                    'name': self.data[i]['name'],
                    'description': self.data[i]['description'],
                    'variables': self.data[i]['variables'],
                    'universe': self.data[i]['universe '].strip(),
                    'distance': float(distances[idx][j])
                }
                for j, i in enumerate(indices[idx])
            ]
        return matches

In [ ]:
# Keywords tailored to your audience
keywords = ['off-road', 'auto', '4x4', 'outdoor', 'camping', 'recreational', 'adventure', 'trail', 'motor']

# Get matched interests
matched_affinity = fuzzy_match_interests(gads_affinity_interests, keywords)
matched_inmarket = fuzzy_match_interests(gads_in_market_interests, keywords)

print("Affinity Matches:")
for match in matched_affinity:
    print(match)

print("\nIn-Market Matches:")
for match in matched_inmarket:
    print(match)

In [ ]:
def filter_relevant_interests(interests_list, keywords):
    relevant = []
    for interest in interests_list:
        if any(keyword.lower() in interest['name'].lower() for keyword in keywords):
            relevant.append(interest)
    return relevant

# Define keywords relevant to offroad/overlanding
keywords = ['off-road', 'auto', '4x4', 'outdoor', 'camping', 'recreational', 'adventure', 'trail', 'motor', 'overland', 'off road']

# Example usage:
relevant_affinity = filter_relevant_interests(gads_affinity_interests, keywords)
relevant_inmarket = filter_relevant_interests(gads_in_market_interests, keywords)

print("Affinity:")
for interest in relevant_affinity:
    print(interest)

print("\nIn-Market:")
for interest in relevant_inmarket:
    print(interest)

In [ ]:
def fetch_custom_segment_geo_insights(client, customer_id, custom_segment_resource_name, geo_location_ids):
    audience_insights_service = client.get_service("AudienceInsightsService")
    googleads_service = client.get_service("GoogleAdsService")

    insights_results = []

    for location_id in geo_location_ids:
        request = client.get_type("GenerateAudienceCompositionInsightsRequest")
        request.customer_id = customer_id

        # Define custom segment audience
        custom_segment_attr = client.get_type("AudienceInsightsAttribute")
        custom_segment_attr.custom_audience = custom_segment_resource_name

        insights_group = client.get_type("InsightsAudienceAttributeGroup")
        insights_group.attributes.append(custom_segment_attr)

        request.audience.topic_audience_combinations.append(insights_group)

        # Location info
        location_info = client.get_type("LocationInfo")
        location_info.geo_target_constant = googleads_service.geo_target_constant_path(location_id)
        request.audience.country_locations.append(location_info)

        request.dimensions.append("GEO_TARGET_REGION")

        response = audience_insights_service.generate_audience_composition_insights(request=request)

        for dimension in response.dimensions:
            insights_results.append({
                "region": dimension.geo_target_constant.name,
                "audience_size": dimension.metrics.audience_size,
                "audience_share": dimension.metrics.audience_share,
            })

    return insights_results

def audience_composition_insights(
    client,
    audience_insights_service,
    googleads_service,
    customer_id,
    location_id,
    user_interest,
    custom_name,
):
    """Returns a collection of attributes represented in an audience of interest.

        Please refere here for more:
        https://developers.google.com/google-ads/api/data/codes-formats

    Args:
        client: an initialized GoogleAdsClient instance.
        audience_insights_service: an initialized AudienceInsightsService
          instance.
        googleads_service: an initialized GoogleAds Service instance.
        customer_id: The customer ID for the audience insights service.
        location_id: The location ID for the audience of interest.
        user_interest: The criterion ID of the category.
        custom_name: custom defined name.
    """
    request = client.get_type("GenerateAudienceCompositionInsightsRequest")
    request.customer_id = customer_id

    insights_info = client.get_type("InsightsAudienceAttributeGroup")
    attributes = client.get_type("AudienceInsightsAttribute")
    attributes.user_interest.user_interest_category = (
        googleads_service.user_interest_path(customer_id, user_interest)
    )

    insights_info.attributes.append(attributes)
    request.audience.topic_audience_combinations.append(insights_info)

    location_info = client.get_type("LocationInfo")
    location_info.geo_target_constant = (
        googleads_service.geo_target_constant_path(location_id)
    )
    request.audience.country_locations.append(location_info)

    request.customer_insights_group = custom_name
    request.dimensions = (
        "AFFINITY_USER_INTEREST",
        "IN_MARKET_USER_INTEREST",
        "YOUTUBE_CHANNEL",
    )
    response = audience_insights_service.generate_audience_composition_insights(
        request=request
    )
    print(response)

keywords = keywords = ["overlanding", "off-road vehicles", "4x4 accessories"]
# geo_location_ids = ["21137", "21176", "9000964"]  # Example: California, Texas, Ontario

us_geo_location_ids =[]

for geo_name, geo_location_id in us_states_google.items():
    # print(geo_name, geo_location_id)
    us_geo_location_ids.append(geo_location_id)

ca_geo_location_ids =[]

for geo_name, geo_location_id in ca_provinces_google.items():
    # print(geo_name, geo_location_id)
    ca_geo_location_ids.append(geo_location_id)

# audience_insights_service = client.get_service("AudienceInsightsService")
# googleads_service = client.get_service("GoogleAdsService")
# results = audience_composition_insights(client, audience_insights_service,  googleads_service, '7759116401', '21138', '91505', 'off-over-empty')

# for r in results:
#     print(f"{r['region']}: {r['audience_size']} ({r['audience_share']:.2%})")

# # us_states ca_provinces

Using census data for land ownership

In [ ]:
# Get the tables for the ACS 5-year data
acs5_tables = c.acs5.tables()
acs5_tables_df = pd.DataFrame(acs5_tables)
acs5_tables_df.head()
acs5_tables_df.loc[acs5_tables_df['name'].str.contains('B25056')]




In [ ]:
searcher = ACSSemanticSearcher(use_gpu=True)
searcher.build_index(data = acs5_tables)

In [ ]:
results = searcher.query(["american housing survey"], top_k=8)

for query, matches in results.items():
    print(f"\nMatches for '{query}':")
    for match in matches:
        print(f"  - Table: {match['name']}")
        print(f"    Description: {match['description']}")
        print(f"    Variables URL: {match['variables']}")
        print(f"    Universe: {match['universe']}")
        print(f"    Distance: {match['distance']:.2f}\n")

In [ ]:

# us_land = pd.read_csv('/content/us_land_ownership.csv')
# us_land.columns.to_list()
# us_land['geo_code'] = us_land['GEOID'].astype(str).str.zfill(2)
# us_land['geo_name'] = us_land['NAME'].str.upper()
# good_cols = ['geo_code', 'geo_name',  'under_0.5_acres',
#  '0.5_to_5_acres',
#  '5_to_10_acres',
#  'over_10_acres',
#  'over_5_acres']



# us_land_good = us_land[good_cols]
# us_land_good.to_csv('us_land_ownership_updated.csv', index=False)


# ca_land_gpd = gpd.read_file('/content/ca_land_ownership-simplified.geojson')

# ca_land = pd.read_csv('/content/us_land_ownership.csv')
# us_land.columns.to_list()
# us_land['geo_code'] = us_land['GEOID'].astype(str).str.zfill(2)
# us_land['geo_name'] = us_land['NAME'].str.upper()
# good_cols = ['geo_code', 'geo_name',  'under_0.5_acres',
#  '0.5_to_5_acres',
#  '5_to_10_acres',
#  'over_10_acres',
#  'over_5_acres']



# us_land_good = us_land[good_cols]
# us_land_good.to_csv('us_land_ownership_updated.csv', index=False)




In [ ]:

acs_variables = {
    "B25056_002E": "Under 1 acre",
    "B25056_003E": "1 to 9.9 acres",
    "B25056_004E": "10 or more acres"
}

# Fetch county-level ACS data (all counties)
acs_data = c.acs5.get(list(acs_variables.keys()), {'for': 'county:*'})

df_acs = pd.DataFrame(acs_data).rename(columns=acs_variables)

# Convert numeric columns
for col in acs_variables.values():
    df_acs[col] = pd.to_numeric(df_acs[col])

# Create full FIPS code
df_acs['FIPS'] = df_acs['state'] + df_acs['county']

df_acs = df_acs[['FIPS', 'Under 1 acre', '1 to 9.9 acres', '10 or more acres']]






In [ ]:

df_rucc = pd.read_excel('/content/Ruralurbancontinuumcodes2023.xlsx', sheet_name='Rural-urban Continuum Code 2023')
df_rucc['FIPS'] = df_rucc['FIPS'].astype(str).str.zfill(5)
df_final = df_acs.merge(df_rucc[['FIPS', 'RUCC_2023', 'Description']], on='FIPS', how='left')

In [ ]:
def classify_area(rucc):
    if rucc <= 3:
        return 'Urban'
    elif rucc <= 6:
        return 'Suburban'
    else:
        return 'Rural'

df_final['AreaType'] = df_final['RUCC_2023'].apply(classify_area)
df_final.head()

In [ ]:
allocation_factors = {
    'Urban': {'under_05': 0.75, '05_to_1': 0.25, '1_to_5': 0.85, '5_to_10': 0.15},
    'Suburban': {'under_05': 0.40, '05_to_1': 0.60, '1_to_5': 0.60, '5_to_10': 0.40},
    'Rural': {'under_05': 0.10, '05_to_1': 0.90, '1_to_5': 0.45, '5_to_10': 0.55}
}

# Correct allocations:
df_final['under_0.5_acres'] = (df_final.apply(
    lambda row: row['Under 1 acre'] * allocation_factors[row['AreaType']]['under_05'], axis=1)).round(0).astype(int)

df_final['0.5_to_5_acres'] = (df_final.apply(
    lambda row: (row['Under 1 acre'] * allocation_factors[row['AreaType']]['05_to_1']) +
                (row['1 to 9.9 acres'] * allocation_factors[row['AreaType']]['1_to_5']),
    axis=1)).round(0).astype(int)

df_final['5_to_10_acres'] = (df_final.apply(
    lambda row: row['1 to 9.9 acres'] * allocation_factors[row['AreaType']]['5_to_10'], axis=1))

df_final['over_10_acres'] = df_final['10 or more acres'].round(0).astype(int)
df_final['STATEFP'] = df_final['FIPS'].str[:2]
df_final['geo_code'] = df_final['STATEFP']
df_final['COUNTY'] = df_final['FIPS'].str[2:]
df_final_summary = df_final.groupby(['geo_code'])[['under_0.5_acres', '0.5_to_5_acres', '5_to_10_acres', 'over_10_acres']].sum().reset_index()
df_final_summary = pd.merge(df_final_summary, df_us_state_pop[['geo_code', 'geo_name']], on='geo_code', how='inner')
df_final_summary[['geo_code', 'geo_name', 'under_0.5_acres', '0.5_to_5_acres', '5_to_10_acres', 'over_10_acres']].to_csv('us_land_ownership_new.csv', index=False)
us_land_ownership_summary = df_final.groupby(['STATEFP'])[['under_0.5_acres', '0.5_to_5_acres', '5_to_10_acres', 'over_10_acres']].sum().reset_index()
us_land_ownership_summary['over_5_acres'] =  (us_land_ownership_summary['5_to_10_acres'] + us_land_ownership_summary['over_10_acres']).round(0).astype(int)
us_land_ownership_summary['under_0.5_acres'] = us_land_ownership_summary['under_0.5_acres'].round().astype(int)
us_land_ownership_summary['0.5_to_5_acres'] = us_land_ownership_summary['0.5_to_5_acres'].round().astype(int)
us_land_ownership_summary
df_final.head()
us_land_ownership = repositioned_us.merge(us_land_ownership_summary[['STATEFP', 'under_0.5_acres', '0.5_to_5_acres', '5_to_10_acres', 'over_10_acres', 'over_5_acres']], on=['STATEFP'], how='left')
#


us_land_ownership.to_csv('us_land_ownership.csv', index=False)

data_col = 'under_0.5_acres'
title_txt = f'Audience Sizing for Land Owners < 0.5 acres'
footer_txt = 'Data Sources: 2022 US American Community Survey and 2023 USDA  Rural-Urban Continuum Codes'
vert_max = us_land_ownership[data_col].max()
file_stub = 'nus_land'

plot_us_map(us_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# us_land_ownership[['STATE_NAME', 'STATEFP', 'PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)

data_col = '0.5_to_5_acres'
title_txt = f'Audience Sizing for Land Owners 0.5 to 5 acres'
footer_txt = 'Data Sources: 2022 US American Community Survey and 2023 USDA  Rural-Urban Continuum Codes'
vert_max = us_land_ownership[data_col].max()
file_stub = 'nus_land'

plot_us_map(us_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# us_land_ownership[['STATE_NAME', 'STATEFP', 'PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


data_col = 'over_5_acres'
title_txt = f'Audience Sizing for Land Owners over 5 acres'
footer_txt = 'Data Sources: 2022 US American Community Survey and 2023 USDA  Rural-Urban Continuum Codes'
vert_max = us_land_ownership[data_col].max()
file_stub = 'nus_land'

plot_us_map(us_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
us_land_ownership['geo_code'] = us_land_ownership['STATEFP']
us_land_ownership_summary = us_land_ownership.groupby(['geo_code'])[['under_0.5_acres', '0.5_to_5_acres', '5_to_10_acres', 'over_10_acres']].sum().reset_index()

# us_land_ownership_summary_states = pd.merge(us_land_ownership_summary, us_land_ownership[['STATEFP', 'STATE_NAME', 'PRENAME', 'PRUID']], on=['STATEFP'], how='left')
us_land_ownership_summary_export = pd.merge(us_land_ownership_summary, us_land_ownership[['geo_code', 'NAME']], on=['geo_code'], how='left')
us_land_ownership_summary_export['geo_name'] = us_land_ownership_summary_export['NAME'].str.upper()
us_land_ownership_summary_export[['geo_code', 'geo_name','under_0.5_acres','0.5_to_5_acres','5_to_10_acres','over_10_acres'  ]].to_csv('us_land_ownership_summary_new.csv', index=False)
# us_land_ownership[['STATE_NAME', 'STATEFP', 'PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


In [ ]:
operator_fields_to_use = ['AREA OPERATED', 'COMMODITY TOTALS - OPERATIONS WITH SALES']
over_5_acre_factor = 0.35
under_250k_factor = 0.9
increaser_factor = 1.3

ca_split_acreage_categories = ['Under 10.00 acres']



ca_over_10_acreage_categories = ['10.00 to 69.99 acres',
    '70.00 to 129.99 acres',
    '130.00 to 179.99 acres',
    '180.00 to 239.99 acres',
    '240.00 to 399.99 acres',
    '400.00 to 559.99 acres',
    '560.00 to 759.99 acres',
    '760.00 to 1,119.99 acres',
    '1,120.00 to 1,599.99 acres',
    '1,600.00 to 2,239.99 acres',
    '2,240.00 to 2,879.99 acres',
    '2,880.00 to 3,519.99 acres',
    '3,520.00 acres and over']


below_250k_categories = [

     '$0',
        '$1 to $9,999',

        '$10,000 to $24,999',
        '$100,000 to $249,999',

        '$25,000 to $49,999',

        '$50,000 to $99,999'

]

above_250k_categories = [

            '$500,000 to $999,999',
            '$250,000 to $499,999',
            '$2,000,000 and over',
        '$1,000,000 to $1,999,999'
]

geo_level='csd'

ca_farmland_df = sc.table_to_df("32100232")
ca_farmland_df['VALUE'] = ca_farmland_df['VALUE'].fillna(0.0).astype(int)
ca_csd_farmland_df = ca_farmland_df.loc[ca_farmland_df['DGUID'].str.len() == 16]
ca_csd_farmland_df_filtered = ca_csd_farmland_df.loc[~(ca_csd_farmland_df['Total farm area distribution'] == 'Total number of farms')]
ca_csd_farmland_df_filtered['geo_name'] = ''




data_col = 'Total farm area distribution'
geo_col='GEO'
pivot1=ca_split_acreage_categories
pivot2=ca_over_10_acreage_categories
pivot1_label='Ops_under_10_acres'
pivot2_label='Ops_10_acres_or_more'
pivot_total_label='Total_Ops'
country='CA'
pivot_col='Total farm area distribution'
values_col='VALUE'
geo_code_col='DGUID'
geo_name_col='geo_name'
fips_col='geo_code'
agg_function='sum'

summary_df = ca_csd_farmland_df_filtered.pivot_table(
    index=geo_code_col,
    columns=pivot_col,
    values=values_col,
    aggfunc=agg_function,
    fill_value=0
).reset_index()


# summary_df['Ops_10_acres_or_more'] = summary_df.copy().reset_index()

# Explicit aggregation for below and above $250k
summary_df[pivot1_label] = summary_df[pivot1].sum(axis=1)
summary_df[pivot2_label] = summary_df[pivot2].sum(axis=1)
# summary_df[geo_code_col] = summary_df[geo_col].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0])
# summary_df[geo_name_col] = summary_df[geo_col].apply(lambda x: x.split('[')[0].strip()).str.upper()

summary_df[pivot_total_label] = summary_df[pivot1_label] + summary_df[pivot2_label]
ca_csd_farmland_df_filtered_summary = pd.merge(ca_csd_farmland_df_filtered, summary_df, on=geo_code_col, how='left')
ca_csd_farmland_df_filtered_summary[[pivot1_label, pivot2_label]] = ca_csd_farmland_df_filtered_summary[[pivot1_label, pivot2_label]].fillna(0.0).round(0).astype(int)

# summary_df[fips_col] = (
#     summary_df[geo_code_col]
#     .str.replace('PR', '', regex=False)  # clearly remove 'PR' if present
#     .str[:2]                             # clearly take the first two characters
#     .astype(int)                         # clearly convert to integer
# )
final_columns = [geo_code_col, pivot1_label, pivot2_label, pivot_total_label]
ca_csd_farmland_df_filtered_summary['geo_code'] = ca_csd_farmland_df_filtered_summary['DGUID']
ca_csd_farmland_df_filtered_summary = ca_csd_farmland_df_filtered_summary[final_columns]
ca_csd_farmland_df_filtered_summary.columns = final_columns
ca_csd_farmland_df_filtered_summary = ca_csd_farmland_df_filtered_summary.copy().sort_values('DGUID').reset_index(drop=True)








# adding rev - but don't care yet
farmland_geo_codes = ca_csd_farmland_df_filtered_summary['DGUID'].drop_duplicates().to_list()
len(farmland_geo_codes)
ca_csd_farmIncome_df = df_canada
ca_csd_farmIncome_df['geo_code'] = ca_csd_farmIncome_df['DGUID'].str[-7:]
# ca_csd_farmIncome_df_filtered = ca_csd_farmIncome_df.loc[~(ca_csd_farmIncome_df['Total farm revenues distribution'] == 'Total number of farms') & (ca_csd_farmIncome_df['geo_code'].isin(farmland_geo_codes))].sort_values(by='geo_code').reset_index(drop=True)
ca_csd_farmIncome_df_filtered = ca_csd_farmIncome_df.loc[~(ca_csd_farmIncome_df['Total farm revenues distribution'] == 'Total number of farms') ].sort_values(by='geo_code').reset_index(drop=True)
ca_csd_farmIncome_df_filtered.head()
data_col = 'Ops_5_acres_or_more'
geo_col='DGUID'
pivot1=ca_split_acreage_categories
pivot2=ca_over_10_acreage_categories
pivot1_label='Ops_below_250k'
pivot2_label='Ops_250k_or_more'

pivot_total_label='Total_Ops'
country='CA'
pivot_col='Total farm revenues distribution'
values_col='VALUE'
geo_code_col='geo_code'
geo_name_col='geo_name'
fips_col='geo_code'
agg_function='sum'
summary_df2 = ca_csd_farmIncome_df_filtered.pivot_table(
    index=geo_code_col,
    columns=pivot_col,
    values='VALUE',
    aggfunc='sum',
    fill_value=0
).reset_index()



# Explicit aggregation for below and above $250k
summary_df2['Ops_below_250k'] = summary_df2[below_250k_categories].sum(axis=1)
summary_df2['Ops_250k_or_more'] = summary_df2[above_250k_categories].sum(axis=1)
summary_df2['Ops_below_250k'] = summary_df2['Ops_below_250k'].round(0).astype(int)
summary_df2.columns.to_list()
# summary_df['geo_code'] = summary_df[geo_col].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0])
# summary_df['geo_name'] = summary_df[geo_col].apply(lambda x: x.split('[')[0].strip()).str.upper()

summary_df2['Total_Ops_with_rev'] = summary_df2['Ops_below_250k'] + summary_df2['Ops_250k_or_more']


# summary_df['PRUID'] = (
#     summary_df['geo_code']
#     .str.replace('PR', '', regex=False)  # clearly remove 'PR' if present
#     .str[:2]                             # clearly take the first two characters
#     .astype(int)                         # clearly convert to integer
# )
final_columns = ['geo_code', 'Ops_below_250k', 'Ops_250k_or_more', 'Total_Ops_with_rev']
summary_df2 = summary_df2[final_columns]
summary_df2.columns = final_columns
summary_df2 = summary_df2.copy().sort_values('geo_code').reset_index(drop=True)


summary_df['geo_code'] = summary_df['DGUID']
total_summary_df = pd.merge(summary_df, summary_df2, on='geo_code', how='left')
total_summary_df



ca_farmland_df = sc.table_to_df("32100232")
ca_farmland_df.columns.to_list()

# use DGUID of 16 for census farm area / 11 for province
# ca_csd_farmland_df = ca_farmland_df.loc[ca_farmland_df['DGUID'].str.len() == 16]
# ca_csd_farmland_df['ca_geo_code'] = ca_farmland_df['DGUID'].str[-7:]
ca_csd_farmland_df['geo_code'] = (ca_farmland_df['DGUID'])
ca_csd_farmland_df['Total farm area distribution'].drop_duplicates().to_list()

ca_csd_farmland_df = ca_csd_farmland_df.loc[ca_csd_farmland_df['Total farm area distribution'] != 'Total number of farms'].copy()

ca_csd_farmland_df['Total farm area distribution'].drop_duplicates().to_list()
ca_csd_farmland_df.head()

ca_csd_farmland_df = ca_csd_farmland_df.pivot_table(
    index='geo_code',
    columns='Total farm area distribution',
    values='VALUE',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Explicit aggregation for below and above $250k
ca_csd_farmland_df['Ops_under_10_acres'] = ca_csd_farmland_df[ca_split_acreage_categories].sum(axis=1).fillna(0.0).astype(int)
ca_csd_farmland_df['Ops_10_acres_or_more'] = ca_csd_farmland_df[ca_over_10_acreage_categories].sum(axis=1).fillna(0.0).astype(int)
ca_csd_farmland_df = ca_csd_farmland_df[['geo_code', 'Ops_under_10_acres', 'Ops_10_acres_or_more']].copy().reset_index(drop=True)
ca_csd_farmland_df.head()

# summary_df['geo_code'] = summary_df['geo_code'].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0])
# summary_df['geo_name'] = summary_df[geo_col].apply(lambda x: x.split('[')[0].strip()).str.upper()

summary_df['Total_Ops'] = summary_df['Ops_below_250k'] + summary_df['Ops_250k_or_more']
summary_df['PRUID'] = (
    summary_df['geo_code']
    .str.replace('PR', '', regex=False)  # clearly remove 'PR' if present
    .str[:2]                             # clearly take the first two characters
    .astype(int)                         # clearly convert to integer
)
final_columns = ['PRUID','geo_name', 'geo_code', 'Ops_below_250k', 'Ops_250k_or_more', 'Total_Ops']
summary_df = summary_df[final_columns]
summary_df.columns = final_columns
summary_df = summary_df.copy().sort_values('PRUID').reset_index(drop=True)

# post income


geo_level='provincial'
ca_csd_farmland_summary_df = categorize_revenues(df_acre_canada, geo_level=geo_level, geo_col='GEO', country_geo_val='Canada [000000000]', above_250k_cats=ca_split_acreage_categories, below_250k_cats=ca_over_10_acreage_categories, country='CA', pivot_col='Total farm area distribution')
ca_csd_farmland_df.head()

#
farm_area_dist_values =  ['Under 10.00 acres',
 '10.00 to 69.99 acres',
 '70.00 to 129.99 acres',
 '130.00 to 179.99 acres',
 '180.00 to 239.99 acres',
 '240.00 to 399.99 acres',
 '400.00 to 559.99 acres',
 '560.00 to 759.99 acres',
 '760.00 to 1,119.99 acres',
 '1,120.00 to 1,599.99 acres',
 '1,600.00 to 2,239.99 acres',
 '2,240.00 to 2,879.99 acres',
 '2,880.00 to 3,519.99 acres',
 '3,520.00 acres and over']

ca_hh_df = sc.table_to_df("9810000201")

ca_hh_df.columns.to_list()
# ca_gaf =  pd.read_csv('/content/2021_92-151_Xb.csv', encoding='ISO-8859-1', dtype=str )
occupied_dwellings_col = 'Population and dwelling counts (13): Private dwellings occupied by usual residents, 2021 [7]'
# ca_rucc = sc.table_to_df("48f544ed-e578-436c-8460-eacb64e61a9d")
ca_rucc = gpd.read_file('/content/gpc_000a11a_e.zip')

ca_rucc['geo_code'] = ca_rucc['PRUID'].astype(str).str.zfill(2) + ca_rucc['PCUID'].astype(str).str.zfill(5)

ca_hh_df['household_count'] = ca_hh_df[occupied_dwellings_col]

ca_hh_df = ca_hh_df[['GEO', 'DGUID', 'household_count']]
ca_hh_df = ca_hh_df.loc[~(ca_hh_df['GEO'] == 'Canada')]
ca_hh_df['GEO'] = ca_hh_df['GEO'].str.strip().str.upper()
ca_hh_df.head()
ca_hh_df.loc[ca_hh_df['GEO'].str.contains('CSD')]
ca_csd_hh_df = ca_hh_df[ca_hh_df['DGUID'].str.len() == 16]
ca_csd_hh_df['PRUID'] = ca_csd_hh_df['DGUID'].str[-7:]
ca_csd_hh_df['geo_code'] = ca_csd_hh_df['PRUID'].str[:2]

ca_csd_hh_df['household_count'] = ca_csd_hh_df['household_count'].fillna(0).astype(int)
ca_csd_hh_df_summary = ca_csd_hh_df.groupby('geo_code')['household_count'].sum().reset_index()
ca_csd_hh_df_summary.head()

merged_csd_dr = pd.merge(ca_csd_hh_df_summary, ca_csd_farmland_df, on='geo_code', how='left',suffixes=('', '_hh')).copy().sort_values(by='geo_code').reset_index(drop=True)
merged_csd_dr['Ops_under_10_acres'] = merged_csd_dr['Ops_under_10_acres'].fillna(0).astype(int)
merged_csd_dr['Ops_10_acres_or_more'] = merged_csd_dr['Ops_10_acres_or_more'].fillna(0).astype(int)
# merged_csd_dr['Total_Ops'] = merged_csd_dr['Total_Ops'].fillna(0).astype(int)
# merged_csd_dr['Total_Ops_with_rev'] = merged_csd_dr['Total_Ops_with_rev'].fillna(0).astype(int)
merged_csd_dr['Ops_5_acres_or_more'] = merged_csd_dr['Ops_5_acres_or_more'].fillna(0).astype(int)
# merged_csd_dr['Ops_below_250k'] = merged_csd_dr['Ops_below_250k'].fillna(0).astype(int)
# merged_csd_dr['Ops_250k_or_more'] = merged_csd_dr['Ops_250k_or_more'].fillna(0).astype(int)
merged_csd_dr['household_count'] = merged_csd_dr['household_count'].fillna(0).astype(int)
merged_csd_dr['HH_after_farms'] = merged_csd_dr['household_count'] - merged_csd_dr['Total_Ops']
merged_csd_dr


26205 2021S05021001214
Division No. 1, Subd. C, Newfoundland and Lab...
ca_gaf['POPCTRRACLASS_CTRPOPRRCLASSE'] = ca_gaf['POPCTRRACLASS_CTRPOPRRCLASSE'].astype(str)
ca_gaf['CSDUID_SDRIDU'] = ca_gaf['CSDUID_SDRIDU'].astype(str)
full_csd_data_classed  = pd.merge(
    merged_csd_dr,
    ca_gaf[['CSDUID_SDRIDU', 'POPCTRRACLASS_CTRPOPRRCLASSE']],
    left_on='geo_code',
    right_on='CSDUID_SDRIDU',
    how='left'
)
full_csd_data_classed.head()


In [ ]:
import pandas as pd
# df = full_csd_data_classed.copy()
# Define farm-to-acreage allocation assumptions explicitly
farm_allocations = {
    '1': 0.75,  # Large Urban
    '2': 0.60,  # Medium Urban
    '3': 0.40,  # Small Urban
    '4': 0.10   # Rural
}

# Define non-farm household acreage allocation explicitly
nonfarm_allocations = {
    '1': {'under_05': 0.80, '05_to_5': 0.15, 'over_5': 0.05},
    '2': {'under_05': 0.60, '05_to_5': 0.30, 'over_5': 0.10},
    '3': {'under_05': 0.40, '05_to_5': 0.40, 'over_5': 0.20},
    '4': {'under_05': 0.15, '05_to_5': 0.35, 'over_5': 0.50}
}

# Force correct data types explicitly
df['POPCTRRACLASS_CTRPOPRRCLASSE'] = df['POPCTRRACLASS_CTRPOPRRCLASSE'].astype(str)
numeric_cols = [
    'household_count', 'HH_after_farms', 'Total_Ops',
    'Ops_under_10_acres', 'Ops_5_acres_or_more'
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
farm_allocations = {
    '1': 0.75,  # Large Urban
    '2': 0.60,  # Medium Urban
    '3': 0.40,  # Small Urban
    '4': 0.10   # Rural
}

nonfarm_allocations = {
    '1': {'under_05': 0.80, '05_to_5': 0.15, 'over_5': 0.05},
    '2': {'under_05': 0.60, '05_to_5': 0.30, 'over_5': 0.10},
    '3': {'under_05': 0.40, '05_to_5': 0.40, 'over_5': 0.20},
    '4': {'under_05': 0.15, '05_to_5': 0.35, 'over_5': 0.50}
}

def calculate_acreage(row):
    urban_class = row['POPCTRRACLASS_CTRPOPRRCLASSE']

    hh_total = row['household_count']
    hh_after_farms = row['HH_after_farms']

    farms_under_5 = row['Ops_under_10_acres']
    farms_over_5 = row['Ops_5_acres_or_more']

    if row['Total_Ops'] > 0:
        # Farms present, allocate explicitly
        farms_under_05 = farms_under_5 * farm_allocations.get(urban_class, 0)
        farms_05_to_5 = farms_under_5 - farms_under_05

        nonfarm_under_05 = hh_after_farms * nonfarm_allocations[urban_class]['under_05']
        nonfarm_05_to_5 = hh_after_farms * nonfarm_allocations[urban_class]['05_to_5']
        nonfarm_over_5 = hh_after_farms * nonfarm_allocations[urban_class]['over_5']

        parcels_under_05 = farms_under_05 + nonfarm_under_05
        parcels_05_to_5 = farms_05_to_5 + nonfarm_05_to_5
        parcels_over_5 = farms_over_5 + nonfarm_over_5
    else:
        # No farm data, allocate clearly using households
        parcels_under_05 = hh_total * nonfarm_allocations[urban_class]['under_05']
        parcels_05_to_5 = hh_total * nonfarm_allocations[urban_class]['05_to_5']
        parcels_over_5 = hh_total * nonfarm_allocations[urban_class]['over_5']

    return pd.Series({
        'parcels_under_0.5_acres': parcels_under_05,
        'parcels_0.5_to_5_acres': parcels_05_to_5,
        'parcels_over_5_acres': parcels_over_5
    })

# Apply calculation explicitly
results_df = df.apply(calculate_acreage, axis=1)

# Concatenate results explicitly and safely back into original dataframe
df = pd.concat([df.reset_index(drop=True), results_df.reset_index(drop=True)], axis=1)
df['PRUID'] = df['geo_code'].str[:2]
df['CSDUID'] = df['geo_code'].str[2:]
# Verify explicitly
print(df[['geo_code', 'parcels_under_0.5_acres', 'parcels_0.5_to_5_acres', 'parcels_over_5_acres']].head())
df.to_csv('ca_full_csd_land_owner_data_classed.csv', index=False)

# Apply clearly to your DataFrame (assuming df is your dataset)
# full_csd_data_classed[['parcels_under_0.5_acres', 'parcels_0.5_to_5_acres', 'parcels_over_5_acres']] = df.apply(calculate_acreage, axis=1)

# # Quick verification clearly displayed
# print(full_csd_data_classed[['geo_code', 'parcels_under_0.5_acres', 'parcels_0.5_to_5_acres', 'parcels_over_5_acres']].head())


In [ ]:
df = pd.read_csv('/content/ca_full_csd_land_owner_data_classed.csv')


df_ca_land_ownership = df.copy()
df_ca_land_ownership.head()

df['geo_code'] = df['geo_code'].astype(str)
df.head()
# shape_dir = '/content/ca_csd_shapes/'
# ca_csd_shape = fetch_unzip_load_shapefile_flexible('https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcsd000b21a_e.zip',shape_dir)
ca_csd_shape = gpd.read_file('/content/ca_csd_simplified.geojson')
ca_csd_shape.to_file('ca_csd_shape.geojson', driver='GeoJSON')
ca_csd_shape.head()
data_col = 'parcels_over_5_acres'
ca_land_ownership = ca_csd_shape.merge(df[['geo_code', 'parcels_under_0.5_acres', 'parcels_0.5_to_5_acres', 'parcels_over_5_acres']], left_on='CSDUID', right_on='geo_code', how='left')
ca_land_ownership.head()
# ca_land_ownership.to_file('ca_land_ownership.geojson', driver='GeoJSON')

title_txt = f'Audience Sizing for CA Land Owners under over 5 acres.'
footer_txt = 'Data Sources: 2021 CA Ag Census, 2021 CA Census'
vert_max = 1000
# ca_land_ownership[data_col].max()
file_stub = 'parcels_over_5_acres'
plot_us_map(ca_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# ca_land_ownership[['PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)

data_col = 'parcels_0.5_to_5_acres'
title_txt = f'Audience Sizing for CA Land Owners over 0.5 to 5 acres.'
footer_txt = 'Data Sources: 2021 CA Ag Census, 2021 CA Census'
vert_max = 1000
# ca_land_ownership[data_col].max()
plot_us_map(ca_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# ca_land_ownership[['PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


data_col = 'parcels_under_0.5_acres'
title_txt = f'Audience Sizing for CA Land Owners under 0.5.'
footer_txt = 'Data Sources: 2021 CA Ag Census, 2021 CA Census'
vert_max = 1000
# ca_land_ownership[data_col].max()
plot_us_map(ca_land_ownership, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
# ca_land_ownership[['PRENAME', 'PRUID',data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)
ca_land_ownership.to_file('ca_land_ownership-simplified.geojson', driver='GeoJSON')


In [ ]:
sc.downloaded_tables

Off roaders & overlanders

In [ ]:

off_roader_keywords = [
    "Off-road trails",
    "4x4 trails",
    "Jeep trails",
    "ATV trails near me",
    "Mud bogging",
    "Rock crawling",
    "Off-road parks",
    "Dirt trails",
    "Off-road events",
    "Off-road driving",
    "Lift kits",
    "Mud tires",
    "Off-road bumpers",
    "Off-road wheels",
    "Winches for Jeeps",
    "Rock sliders",
    "Suspension lifts",
    "Off-road lights",
    "Snorkel kit",
    "Differential lockers",
    "Jeep Wrangler mods",
    "Ford Bronco off-road",
    "Toyota Tacoma off-road",
    "ATV off-road gear",
    "Side-by-side (UTV) accessories",
    "Polaris Ranger accessories",
    "Rubicon Trail"
]
google_geos = fetch_google_ads_geo_targets(client)

# google_geos = pd.read_csv('/content/geotargets-2025-04-01.csv')
google_us_states = google_geos[((google_geos['country_code'] == 'US') & (google_geos['target_type'] == 'State'))].copy().sort_values('name').reset_index(drop=True)
google_us_states = google_us_states[['name', 'id']].sort_values('name').reset_index(drop=True)
google_us_states['name'] = google_us_states['name'].str.upper()
# google_us_states.rename(columns={'Name': 'NAME', 'Criteria ID': 'geo_id'}, inplace=True)
us_states = google_us_states.set_index('name')['id'].to_dict()
google_ca_provinces = google_geos[((google_geos['country_code'] == 'CA') & (google_geos['target_type'] == 'Province'))].copy().sort_values('name').reset_index(drop=True)
google_ca_provinces['name'] = google_ca_provinces['name'].str.upper()
google_ca_provinces = google_ca_provinces[['name', 'id']].sort_values('name').reset_index(drop=True)
ca_provinces = google_ca_provinces.set_index('name')['id'].to_dict()


# df_estimates = get_keyword_estimates(client, keywords, us_states)

overlander_keywords = [
    "Overlanding routes",
    "Overlanding gear",
    "Roof top tents",
    "Expedition vehicles",
    "Vehicle camping",
    "Self-reliant travel",
    "Off-grid vehicle setups",
    "Long-distance off-road travel",
    "Adventure travel vehicles",
    "Overlanding trips",
    "Roof racks for camping",
    "Overlanding trailers",
    "Vehicle fridge/freezers",
    "Vehicle storage solutions",
    "Portable power stations",
    "Recovery gear overlanding",
    "Vehicle awnings",
    "Off-grid vehicle showers",
    "Portable water filtration systems",
    "Overlanding communication devices",
    "Overland Expo",
    "Overland Bound",
    "Expedition Portal",
    "Off-grid camping communities",
    "EarthRoamer",
    "Adventure vans",
    "Land Rover Defender overlanding",
    "Toyota Land Cruiser overlanding"
]




In [ ]:


acs_variables = {
    "B01003_001E": "total_population"
}
acs_pop_data = c.acs5.get(list(acs_variables.keys()), {'for': 'county:*'})

df_acs_pop = pd.DataFrame(acs_pop_data).rename(columns=acs_variables)

# Convert numeric columns
for col in acs_variables.values():
    df_acs_pop[col] = pd.to_numeric(df_acs_pop[col])

# Create full FIPS code
df_acs_pop['FIPS'] = df_acs_pop['state'] + df_acs_pop['county']
df_acs_pop['total_population'] = df_acs_pop['total_population'].astype(int)

df_us_state_pop = df_acs_pop[['state', 'total_population']].groupby('state').sum().sort_values(by='state').reset_index()
df_us_state_pop['total_population'] = df_us_state_pop['total_population'].astype(int)
import us

# Assuming your dataframe is named 'us_pop_df' and has a column 'state' with abbreviations

df_us_state_pop_clean = convert_state_abbrev_to_full(df_us_state_pop)
df_us_state_pop_clean['geo_name'] = df_us_state_pop_clean['state_full'].str.upper()
df_us_state_pop_clean['geo_code'] = 'us-' + df_us_state_pop_clean['state']
df_us_state_pop_clean
df_us_state_pop = df_us_state_pop_clean[['geo_code', 'geo_name', 'total_population']].copy()
df_us_state_pop
ca_pop_df = sc.table_to_df("9810000201") # canadian population
ca_pop_df['Population and dwelling counts (13): Private dwellings occupied by usual residents, 2021 [7]'] = ca_pop_df['Population and dwelling counts (13): Private dwellings occupied by usual residents, 2021 [7]'].fillna(0).astype(int)
ca_province_pop_df = ca_pop_df[ca_pop_df['DGUID'].str.len() == 11].copy()
ca_province_pop_df['geo_code'] = 'ca-' +  ca_province_pop_df['DGUID'].str[-2:]
ca_province_pop_df['total_population'] = ca_province_pop_df['Population and dwelling counts (13): Private dwellings occupied by usual residents, 2021 [7]']
ca_province_pop_df['geo_name'] = ca_province_pop_df['GEO'].str.upper()
ca_province_pop_df = ca_province_pop_df[['geo_code','geo_name', 'total_population']].sort_values(by='geo_code').copy().reset_index(drop=True)
ca_province_pop_df
na_population_by_region = pd.concat([df_us_state_pop, ca_province_pop_df], ignore_index=True)



In [ ]:
# us_small_search_trends = load_trends_from_search_api(us_under_250k_diy_keywords, search_api_io_key)
kws = off_roader_keywords

na_locales = {**us_states, **ca_provinces}
paid_trends = get_exact_keyword_volumes(client, kws, na_locales)
paid_trends['geo_name'] = paid_trends['state']
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].fillna(0).astype(int)
paid_trends.head()
# paid_trends['keyword'].drop_duplicates().sort_values()
# paid_trends.to_csv('na_off_roader_paid_estimates.csv', index=False)


off_roaders_audience_df = estimate_audience_distribution(paid_trends, na_population_by_region)
off_roaders_audience_df.to_csv('na_off_roader_audience_estimates.csv', index=False)

kws = overlander_keywords
paid_trends = get_exact_keyword_volumes(client, kws, na_locales)
paid_trends['geo_name'] = paid_trends['state']
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].fillna(0).astype(int)

overlanders_audience_df = estimate_audience_distribution(paid_trends, na_population_by_region, total_audience=12000000)
overlanders_audience_df.to_csv('na_overlanders_audience_estimates.csv', index=False)

# us_small_search_trends = get_trends(kws, geo='US')
us_small_search_trends = load_trends_from_search_api_parallel(kws, search_api_io_key)
# get_trends


ca_small_search_trends = load_trends_from_search_api(ca_under_250k_diy_keywords, search_api_io_key, geo='CA')
ca_small_paid = get_keyword_estimates(client, ca_under_250k_diy_keywords, ca_provinces)

us_small_search_trends_flat = us_small_search_trends.reset_index()
us_small_op_trends_prepped = prep_trends_data(us_small_search_trends_flat, us_all_regions, neutral_factor=0.5)
us_small_op_trends_prepped
us_small_op_paid_prepped = prep_paid_search_data(us_small_paid, us_all_regions, neutral_factor=0.5)
us_small_op_paid_prepped
counts_col='Ops_below_250k'
us_small_op_audiences = add_us_stats_and_geos(repositioned_us, us_small_op_trends_prepped, state_summary, us_small_op_paid_prepped, counts_col=counts_col)
sus_mall_op_audiences = us_small_op_audiences.to_crs(epsg=5070)
us_small_op_audiences.loc[us_small_op_audiences['NAME'] == 'PUERTO RICO']
vert_max = us_small_op_audiences['adjusted_audience_Ops_below_250k'].max()
data_col = 'adjusted_audience_Ops_below_250k'
file_stub = 'us_diy_under250k'
leg_kwds_dict={'label': "Estimated Audience", 'orientation': "vertical", 'shrink': 0.7}
title_txt = f'Audience Sizing for DIY Interest: US Farm Operations < $250k'
footer_txt = 'Data Sources: USDA, Google Ads API, Google Trends (Data normalized individually then summed)'

plot_us_map(us_small_op_audiences, density_cmap, data_col, leg_kwds_dict, title_txt, footer_txt, vert_max)
us_small_op_audiences[['NAME', 'STATEFP',counts_col,data_col]].to_csv(f'{file_stub}_adjusted_audience_by_state.csv', index=False)


scratchpad

In [ ]:
us_landowner_df = pd.read_csv('/content/us_land_ownership.csv')
us_landowner_df['under_0.5_acres'].sum()
us_landowner_df['0.5_to_5_acres'].sum()
us_landowner_df['over_5_acres'].sum()
us_landowner_df['under_0.5_acres'].sum() + us_landowner_df['0.5_to_5_acres'].sum() + us_landowner_df['over_5_acres'].sum()
132000000 * .65
# 29,644,595 under .5
# 12,676,255 0.5 to 5
# 334,649
# 132,000,000
# 85,800,000
# 42,655,499

labor counts

In [ ]:
os.environ.['BLS_API_KEY'] = userdata.get('us_bol_stats_api_key')

In [ ]:
import requests
import pandas as pd

API_KEY = userdata.get('us_bol_stats_api_key')
API_KEY = '69c108d10be243598c9cbd5a31a3592d'

In [ ]:
import us

def correct_series_id(state_fips, naics, ownership='5'):
    return f"ENU{state_fips:02d}00000{ownership}{naics}"

series = []
state_map = {}

naics_codes = {
    '561730': '5',  # Private
    '811210': '5',  # Private
    '811310': '5',  # Private
    '921190': '0'   # Government (All ownership)
}

for state in us.states.STATES:
    fips = int(state.fips)
    state_map[f"{fips:02d}"] = state.name
    for naics, ownership in naics_codes.items():
        series_id = correct_series_id(fips, naics, ownership)
        series.append(series_id)
        print(f"Validated Series ID: {series_id} for {state.name}")

In [ ]:
import requests
import os
import us
import pandas as pd

API_URL = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

naics_codes = {
    '561730': '5',  # Private
    '811210': '5',  # Private
    '811310': '5',  # Private
    '921190': '0'   # Government (All ownership)
}

# Helper function to build series ID
def qcew_series_id(naics, state_fips):
    return f'ENU{state_fips:02d}000' + '0' + f'{int(naics):06d}'

def correct_series_id(state_fips, naics, ownership='5'):
    return f"ENU{state_fips:02d}00000{ownership}{naics}"



naics_codes = ['561730', '811210', '811310', '921190']

series = []
state_map = {}

for state in us.states.STATES:
    fips = int(state.fips)
    state_map[f"{fips:02d}"] = state.name
    for naics in naics_codes:
        ownership = '5' if naics != '921190' else '0'  # Government support is usually public (ownership=0)
        series_id = f"ENU{fips:02d}0000{ownership}{naics}"
        series.append(series_id)
        print(f"Generated Correct Series ID: {series_id} for {state.name}")

print("Requesting data from BLS...")
response = requests.post(API_URL, json={
    "seriesid": series[:50],  # BLS API allows up to 50 series per request
    "startyear": "2023",
    "endyear": "2023",
    "registrationkey": API_KEY
})

results = response.json()

response = requests.post(API_URL, json={
    "seriesid": ['ENU48000561730'],  # BLS API allows up to 50 series per request
    "startyear": "2023",
    "endyear": "2023",
    "registrationkey": API_KEY
})

results = response.json()
results


# Inspect response thoroughly
print(results)

# Parsing Results if data is returned
records = []
for s in results.get('Results', {}).get('series', []):
    series_id = s['seriesID']
    state_fips = series_id[3:5]
    naics_code = series_id[-6:]

    for data_point in s.get('data', []):
        records.append({
            'State': state_map.get(state_fips, 'Unknown'),
            'NAICS': naics_code,
            'Year': data_point['year'],
            'Employment': data_point['value']
        })

df = pd.DataFrame(records)
print(df.head())

In [ ]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2022", "endyear":"2023"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
json_data
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

dol_df = pd.read_csv('/content/CUUR0000SA0.txt', sep="\t")
dol_df.head()


In [ ]:
import urllib.request
import pandas as pd
from io import StringIO
import us

def fetch_qcew_employment_all_states(year, qtr, industry_codes):
    all_states_data = []

    for state in us.states.STATES:
        area_fips = state.fips + '000'
        url = f"http://data.bls.gov/cew/data/api/{year}/{qtr.lower()}/area/{area_fips}.csv"

        try:
            with urllib.request.urlopen(url) as response:
                csv_data = response.read().decode('utf-8')

            df = pd.read_csv(StringIO(csv_data))
            df_filtered = df[df['industry_code'].isin(industry_codes)]

            df_filtered['state'] = state.name
            all_states_data.append(df_filtered[[
                'state', 'area_fips', 'own_code', 'industry_code', 'agglvl_code',
                'size_code', 'year', 'qtr', 'annual_avg_estabs', 'annual_avg_emplvl'
            ]])

        except Exception as e:
            print(f"Failed to fetch data for {state.name}: {e}")

    return pd.concat(all_states_data, ignore_index=True)

# Example usage:
industry_codes = ["561730", "811210", "811310", "921190"]
all_states_employment = fetch_qcew_employment_all_states("2024", "a", industry_codes)
print(all_states_employment.head(20))

fixing ca lots

In [ ]:
ca_land = pd.read_csv('/content/ca_full_csd_land_owner_data_classed.csv')

ca_land_unique = ca_land.drop_duplicates(subset=['DGUID'])
ca_land_unique.columns.to_list()
ca_land_summary = (
    ca_land_unique
    .groupby('PRUID', as_index=False)
    .agg({
        'parcels_under_0.5_acres': 'sum',
        'parcels_0.5_to_5_acres': 'sum',
        'parcels_over_5_acres': 'sum'
    })
)
ca_land_summary['PRUID'] = ca_land_summary['PRUID'].astype(str)
ca_land_summary['PRUID'] = 'ca-' + ca_land_summary['PRUID']

ca_land_summary = pd.merge(ca_land_summary,ca_province_pop_df[['geo_code', 'geo_name']], left_on='PRUID', right_on='geo_code', how='left')
ca_land_summary.to_csv('ca_land_ownership_updated.csv', index=False)

In [ ]:
all_states_employment.loc[all_states_employment['state'] == 'Colorado']

In [ ]:
landscaping_operations = all_states_employment.loc[all_states_employment['industry_code'] == '561730']
print(landscaping_operations['annual_avg_estabs'].sum())
landscaping_operations_summary = landscaping_operations.groupby('state').agg({'annual_avg_estabs': 'sum', 'annual_avg_emplvl': 'sum'}).reset_index()
landscaping_operations_summary.to_csv('us_landscaping_operations_audience_summary.csv', index=False)

government_support = all_states_employment.loc[all_states_employment['industry_code'] == '921190']
print(government_support['annual_avg_emplvl'].sum())
government_support_summary = government_support.groupby('state').agg({'annual_avg_estabs': 'sum', 'annual_avg_emplvl': 'sum'}).reset_index()
government_support_summary.to_csv('us_government_support_audience_summary.csv', index=False)
diesel_and_precision_repair = all_states_employment.loc[all_states_employment['industry_code'].isin(['811210','811310']) ]
print(diesel_and_precision_repair['annual_avg_emplvl'].sum())
diesel_and_precision_repair_summary = diesel_and_precision_repair.groupby('state').agg({'annual_avg_estabs': 'sum', 'annual_avg_emplvl': 'sum'}).reset_index()
diesel_and_precision_repair_summary.to_csv('us_diesel_and_precision_repair_audience_summary.csv', index=False)

# landscaping 123772 902787
# government support 436704 - 8800 establishments
# diesel and precision repair 345420

In [ ]:
# canada
ca_biz_df = sc.table_to_df("33‑10‑0806‑01") # canadian employment 9810000201 14-10-0320-02 1410022301
ca_biz_df['PRUID'] = ca_biz_df['DGUID'].str[-2:]
ca_biz_df['geo_name'] = ca_biz_df['GEO'].str.upper()
ca_provinces = ca_biz_df[['PRUID', 'geo_name']].drop_duplicates().sort_values('PRUID').reset_index(drop=True)
ca_provinces.to_csv('ca_provinces.csv', index=False)
ca_biz_df['North American Industry Classification System (NAICS)'].drop_duplicates().sort_values()
ca_biz_df.loc[ca_biz_df['North American Industry Classification System (NAICS)'].str.contains('561730')]
ca_province_landscapers = ca_biz_df.loc[(ca_biz_df['North American Industry Classification System (NAICS)'].str.contains('561730')) & (ca_biz_df['DGUID'].str.len() == 11) & (ca_biz_df['Employment size'] == 'Total, with employees')].copy().reset_index(drop=True)
ca_province_landscapers['VALUE_NUMERIC'] = ca_province_landscapers['VALUE'].fillna(0.0).round(0).astype(int)
# ca_province_landscapers['PRUID'] = ca_province_landscapers['DGUID'].str[-2:]

ca_province_landscapers = ca_province_landscapers.rename(columns={'VALUE_NUMERIC': 'annual_avg_estabs', 'PRUID' : 'geo_code'})
columns = ['geo_code', 'geo_name', 'annual_avg_estabs']
ca_province_landscapers = ca_province_landscapers[columns]
ca_province_landscapers.to_csv('ca_landscaping_operations_audience_summary.csv', index=False)
#
ca_province_procurement = ca_biz_df.loc[(ca_biz_df['North American Industry Classification System (NAICS)'].str.contains('921190')) & (ca_biz_df['DGUID'].str.len() == 11) & (ca_biz_df['Employment size'] == 'Total, with employees')].copy().reset_index(drop=True)

ca_province_diesel = ca_biz_df.loc[((ca_biz_df['North American Industry Classification System (NAICS)'].str.contains('811210'))| (ca_biz_df['North American Industry Classification System (NAICS)'].str.contains('811310'))) & (ca_biz_df['DGUID'].str.len() == 11) & (ca_biz_df['Employment size'] == 'Total, with employees')].copy().reset_index(drop=True)

# https://www150.statcan.gc.ca/n1/pub/71m0001x/2021001/2025-05-CSV.zip
try2 = pd.read_csv('/content/9810045201_databaseLoadingData (1).csv')
try2.columns
# ca_emp_df = sc.table_to_df("98-10-0452-01") # canadian employment 9810000201 14-10-0320-02 1410022301 14-10-0023-01
ca_emp_df.head()
ca_emp_df.loc[ca_emp_df['National Occupational Classification (NOC)'].str.contains('8255')]
ca_emp_df['National Occupational Classification (NOC)'].drop_duplicates().sort_values()

In [ ]:
import pandas as pd

# Load your microdata (assuming CSV or similar)
lfs_df = try2

# Filter employed in NOC 8255
noc_8255_employed = lfs_df[
    (lfs_df['NOC_43'] == '8255')
]

# Group by province and sum weights to get employment counts
employment_counts = (
    noc_8255_employed
    .groupby('PROV')['FINALWT']
    .sum()
    .reset_index()
)

print(employment_counts)

Working on consumer equipment diyers - "Where's Barry?"

In [ ]:
consumer_lawn_garden_keywords = [
    # Lawn Mower & Lawn Tractor Maintenance
    "lawn mower repair",
    "lawn tractor maintenance",
    "riding mower troubleshooting",
    "lawn tractor belt replacement",
    "lawn mower blades replacement",
    "lawn tractor battery replacement",
    "riding mower tire change",
    "garden tractor oil change",
    "lawn tractor engine maintenance",
    "mower deck leveling",

    # Compact & Garden Tractor Repairs
    "garden tractor attachments",
    "small tractor DIY repair",
    "garden tractor electrical issues",
    "compact tractor engine troubleshooting",
    "garden tractor carburetor cleaning",
    "tractor spark plug replacement",
    "tractor air filter replacement",
    "garden tractor fuel system repair",
    "tractor starter replacement",
    "garden tractor transmission repair",

    # Utility Vehicles ("Gator" type vehicles)
    "utility vehicle DIY repair",
    "Gator oil change",
    "side by side maintenance",
    "utility vehicle tire replacement",
    "UTV battery issues",
    "Gator transmission troubleshooting",
    "side by side brake repair",
    "UTV electrical repair",
    "utility vehicle accessory installation",
    "side by side drive belt replacement",

    # Parts & Manuals (Online shopping intent)
    "buy lawn mower parts online",
    "garden tractor parts online",
    "utility vehicle accessories online",
    "John Deere Gator parts online",
    "riding mower replacement parts",
    "tractor service manuals online",
    "UTV repair manuals",
    "garden tractor aftermarket parts",
    "lawn tractor oil change kit",
    "utility vehicle service kits",

    # Seasonal & General DIY
    "winterize garden tractor",
    "utility vehicle winter preparation",
    "lawn tractor storage prep",
    "tractor seasonal maintenance",
    "Gator winter maintenance",
    "garden tractor DIY videos",
    "small engine DIY repairs",
    "lawn equipment troubleshooting guide",
    "utility vehicle maintenance schedule",
    "lawn mower seasonal upkeep",
]


df_us_state_pop
ca_province_pop_df
# kws = us_high_revenue_keywords
kws = consumer_lawn_garden_keywords

paid_trends = get_exact_keyword_volumes(client, kws, us_states_google)
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].astype(int)
paid_trends['avg_monthly_searches'].sum()
paid_trends['geo_name'] = paid_trends['state']
us_consumer_repair = paid_trends
us_consumer_repair['avg_monthly_searches'].sum()

paid_trends = get_exact_keyword_volumes(client, kws, ca_provinces_google)
paid_trends['avg_monthly_searches'] = paid_trends['avg_monthly_searches'].astype(int)
paid_trends['avg_monthly_searches'].sum()
paid_trends['geo_name'] = paid_trends['state']
ca_consumer_repair = paid_trends
ca_consumer_repair['avg_monthly_searches'].sum()


us_consumer_diy_spi_index = estimate_search_population_indexes(us_consumer_repair, df_us_state_pop, population_col='adults_18_to_64')
us_consumer_diy_repair_audience = pd.merge(df_us_state_pop, us_consumer_diy_spi_index[['geo_code','SPI', 'normalized_SPI'] ], on='geo_code', suffixes=('', '_spi'))
us_consumer_diy_repair_audience['modeled_audience_size'] = us_consumer_diy_repair_audience['adults_18_to_64'] * us_consumer_diy_repair_audience['normalized_SPI']
us_consumer_diy_repair_audience['modeled_audience_size'] = us_consumer_diy_repair_audience['modeled_audience_size'].astype(int)
us_consumer_diy_repair_audience.to_csv('us_consumer_repair_audience_estimates.csv', index=False)
# 47350.0 3,444,187
ca_consumer_diy_spi_index = estimate_search_population_indexes(ca_consumer_repair, ca_province_pop_df, population_col='adults_18_to_64')
ca_consumer_diy_repair_audience = pd.merge(ca_province_pop_df, ca_consumer_diy_spi_index[['geo_code','SPI', 'normalized_SPI'] ], on='geo_code', suffixes=('', '_spi'))
ca_consumer_diy_repair_audience['modeled_audience_size'] = ca_consumer_diy_repair_audience['adults_18_to_64'] * us_consumer_diy_repair_audience['normalized_SPI']
ca_consumer_diy_repair_audience['modeled_audience_size'] = ca_consumer_diy_repair_audience['modeled_audience_size'].astype(int)
ca_consumer_diy_repair_audience.to_csv('ca_consumer_repair_audience_estimates.csv', index=False)
ca_consumer_diy_repair_audience['modeled_audience_size'].sum()
